**项目：我国研究区域火灾活动的主导时空模式与季节性**

本节目标：
- 明确研究数据与处理流程。
- 构建从原始卫星火点数据到时空统计与可视化的分析管线。
- 产出两类结论：
  1) 全部火灾事件的季节性（按月/季节的时间分布）。
  2) 地理分布（空间聚集、县域尺度分布）。

数据来源与位置：
- 原始数据在 `raw_data/` 下：
  - 卫星火点数据：`raw_data/satellite_fire_data/modis_*.csv`（2010–2019）。
  - 中国县级边界：`raw_data/chn_county/chn_county.shp`。

方法概述：
- 读取并合并 2010–2019 年 MODIS 火点数据；标准化经纬度、时间字段，生成 GeoDataFrame。
- 计算月度与季节聚合，绘制折线图与年×月热力图，刻画季节性。
- 读取县界矢量，做空间连接，统计县域总火点数与年尺度火点数，绘制县域火点分布图。
- 将关键汇总表导出至 `results/` 目录。

注意：
- 代码对常见列名做了鲁棒匹配（latitude/lat、longitude/lon、acq_date/date 等）。
- 若 shapefile 或某些年份数据缺失，流程会给出提示但尽量继续运行。

In [ ]:
# 任务1输出目录改写（将结果写入 results/task1）
from pathlib import Path
import os
try:
    ROOT = Path.cwd()
    RESULTS_TASK1 = ROOT/'results'/'task1'
    RESULTS_TASK1.mkdir(parents=True, exist_ok=True)
    # 若已有全局 RESULTS_DIR（任务1可能使用它保存），则重定向到 task1
    if 'RESULTS_DIR' in globals():
        RESULTS_DIR = RESULTS_TASK1
    else:
        RESULTS_DIR = RESULTS_TASK1
    print('Task1 输出目录:', RESULTS_DIR)
except Exception as e:
    print('Task1 输出目录改写失败:', e)

In [ ]:
# 基础库导入与全局设置
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

import matplotlib.pyplot as plt
import seaborn as sns

# 统一绘图风格
sns.set(style="whitegrid", context="notebook")
plt.rcParams['figure.dpi'] = 120
plt.rcParams['axes.unicode_minus'] = False

# 字体设置：自动使用系统中可用的中文字体
from matplotlib import font_manager
def set_chinese_font(preferred=('Microsoft YaHei','SimHei','SimSun','Noto Sans CJK SC','Source Han Sans SC','WenQuanYi Zen Hei')):
    available = set(f.name for f in font_manager.fontManager.ttflist)
    for name in preferred:
        if name in available:
            plt.rcParams['font.sans-serif'] = [name]
            print('使用中文字体:', name)
            return name
    # 兜底：保留默认 sans-serif，并提示
    plt.rcParams['font.sans-serif'] = plt.rcParams.get('font.sans-serif', ['DejaVu Sans'])
    print('未检测到常见中文字体，可能导致中文无法显示。请在系统安装如 SimHei 或 Microsoft YaHei。')
set_chinese_font()

print("Libraries imported and font configured.")

In [ ]:
# 路径配置与辅助函数
# 自动探测仓库根目录（向上查找包含 raw_data 的目录）
from pathlib import Path

def detect_repo_root(start: Path = Path.cwd()) -> Path:
    for p in [start] + list(start.parents):
        if (p / 'raw_data').exists():
            return p
    # 兜底：若当前在 script/ 下
    return start.parent if start.name == 'script' else start

REPO_ROOT = detect_repo_root()
RAW_DIR = REPO_ROOT / 'raw_data'
FIRE_DIR = RAW_DIR / 'satellite_fire_data'
COUNTY_SHP = RAW_DIR / 'chn_county' / 'chn_county.shp'
RESULTS_DIR = REPO_ROOT / 'results'
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print('Repo root:', REPO_ROOT)
print('Fire data dir exists:', FIRE_DIR.exists())
print('County shp exists:', COUNTY_SHP.exists())
print('Results dir:', RESULTS_DIR)

# 将月份映射到季节（可根据研究需要调整）
MONTH2SEASON = {
    12: 'Winter', 1: 'Winter', 2: 'Winter',
     3: 'Spring', 4: 'Spring', 5: 'Spring',
     6: 'Summer', 7: 'Summer', 8: 'Summer',
     9: 'Autumn', 10: 'Autumn', 11: 'Autumn'
}

def find_column(cols, candidates):
    """在列名中查找第一个匹配项（大小写不敏感）。
    candidates: 如 ['latitude','lat']
    返回匹配到的列名或 None。
    """
    for cand in candidates:
        for col in cols:
            if col.lower() == cand.lower():
                return col
    return None

def build_datetime(df):
    """从常见字段构造 datetime 列，并补充 year/month/day/season。"""
    import pandas as pd
    # 常见字段：acq_date(YYYY-MM-DD)、acq_time(HHMM)
    date_col = find_column(df.columns, ['acq_date', 'date', 'acq-date'])
    time_col = find_column(df.columns, ['acq_time', 'time', 'acq-time'])
    dt = None
    if date_col is not None and time_col is not None:
        # 将 HHMM 转换为 HH:MM
        def _fmt_time(t):
            try:
                s = str(int(t)).zfill(4)
                return f"{s[:2]}:{s[2:]}"
            except Exception:
                return None
        times = df[time_col].apply(_fmt_time)
        dt = pd.to_datetime(df[date_col].astype(str) + ' ' + times.astype(str), errors='coerce')
    elif date_col is not None:
        dt = pd.to_datetime(df[date_col], errors='coerce')
    else:
        # 有的产品用 'timestamp'
        ts_col = find_column(df.columns, ['timestamp'])
        if ts_col is not None:
            dt = pd.to_datetime(df[ts_col], errors='coerce')
    if dt is None:
        raise ValueError('未能识别日期/时间列，请检查原始 CSV 列名。')

    df['datetime'] = dt
    df['date'] = df['datetime'].dt.date
    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    df['day'] = df['datetime'].dt.day
    df['season'] = df['month'].map(MONTH2SEASON)
    return df

def to_geodf(df):
    """将 DataFrame 转为 WGS84 的 GeoDataFrame。"""
    import geopandas as gpd
    from shapely.geometry import Point
    lon_col = find_column(df.columns, ['longitude', 'lon', 'long'])
    lat_col = find_column(df.columns, ['latitude', 'lat'])
    if lon_col is None or lat_col is None:
        raise ValueError('未找到经纬度列（longitude/latitude 或 lon/lat）。')
    geometry = [Point(xy) for xy in zip(df[lon_col].astype(float), df[lat_col].astype(float))]
    gdf = gpd.GeoDataFrame(df.copy(), geometry=geometry, crs='EPSG:4326')
    return gdf

print('Helpers ready.')

In [ ]:
# 读取并合并 2010–2019 年卫星火点数据
from glob import glob

csv_paths = sorted(glob(str(FIRE_DIR / 'modis_*_china.csv')))
if not csv_paths:
    print('未找到 CSV：', FIRE_DIR)
else:
    print(f'发现 {len(csv_paths)} 个年份文件')

frames = []
for p in csv_paths:
    try:
        df = pd.read_csv(p)
        # 构造时间字段
        df = build_datetime(df)
        # 限定合理经纬度范围（粗滤除异常值）
        lon_col = find_column(df.columns, ['longitude', 'lon', 'long'])
        lat_col = find_column(df.columns, ['latitude', 'lat'])
        if lon_col and lat_col:
            df = df[(df[lon_col].between(70, 140)) & (df[lat_col].between(15, 55))]
        # 添加年份来源（从文件名解析）
        # 文件命名形如 modis_2010_china.csv
        try:
            year_in_name = int(Path(p).stem.split('_')[1])
            df['year_from_fname'] = year_in_name
        except Exception:
            df['year_from_fname'] = df['year']
        frames.append(df)
    except Exception as e:
        print(f'读取失败 {p}: {e}')

fire_df = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
print('合并后记录数:', len(fire_df))

# 转为 GeoDataFrame
if not fire_df.empty:
    fire_gdf = to_geodf(fire_df)
    # 确保 CRS 为 WGS84
    fire_gdf = fire_gdf.set_crs('EPSG:4326', allow_override=True)
    display(fire_gdf.head(3))
else:
    fire_gdf = gpd.GeoDataFrame(columns=['datetime','year','month','season','geometry'], geometry='geometry', crs='EPSG:4326')

print('GeoDataFrame ready:', len(fire_gdf))

In [ ]:
# 数据概览与完整性检查
if len(fire_gdf) == 0:
    print('没有有效火点记录，后续单元可能无法绘图。请检查 raw_data/satellite_fire_data 是否存在并包含 CSV。')
else:
    # 时间范围
    tmin, tmax = fire_gdf['datetime'].min(), fire_gdf['datetime'].max()
    print(f"时间范围: {tmin} — {tmax}")
    print('年份分布:')
    display(fire_gdf['year'].value_counts().sort_index())

    # 缺失情况
    missing = fire_gdf[['datetime','year','month','season','geometry']].isna().mean().round(4)
    print('关键字段缺失率:')
    display(missing)

    # 简要统计
    print('火点总数:', len(fire_gdf))
    print('唯一日期数:', fire_gdf['date'].nunique())

    # 示例前几行
    display(fire_gdf.head())

# 季节性与时间分布：思路说明
本节从时间维度刻画火灾活动的主导模式：
- 按月聚合，观察跨年总体的月度强度变化（折线图）。
- 计算“年×月”的二维分布热力图，看每年内的季节高峰是否一致或有迁移。
- 按季节（春夏秋冬）汇总并比较总量。

In [ ]:
# 月度聚合与折线图
if len(fire_gdf) > 0:
    monthly = fire_gdf.groupby('month').size().rename('count').reset_index()
    monthly = monthly.sort_values('month')

    plt.figure(figsize=(8,4))
    sns.lineplot(data=monthly, x='month', y='count', marker='o')
    plt.title('全部年份合并的月度火点数分布')
    plt.xlabel('月份')
    plt.ylabel('火点数')
    plt.xticks(range(1,13))
    plt.tight_layout()
    plt.show()

    # 保存 CSV 与图
    monthly_out = RESULTS_DIR / 'all_years_monthly_fire_counts.csv'
    monthly.to_csv(monthly_out, index=False)
    print('保存:', monthly_out)
else:
    print('无数据，跳过月度聚合。')

In [ ]:
# 年 × 月 热力图（观察年内季节峰值与跨年差异）
if len(fire_gdf) > 0:
    year_month = fire_gdf.groupby(['year','month']).size().rename('count').reset_index()
    # 仅保留合理年份（从文件名或数据解析获得）
    if 'year_from_fname' in fire_gdf.columns:
        year_month = fire_gdf.groupby(['year_from_fname','month']).size().rename('count').reset_index()
        year_month = year_month.rename(columns={'year_from_fname':'year'})

    pivot = year_month.pivot(index='year', columns='month', values='count').fillna(0)

    plt.figure(figsize=(10, 6))
    sns.heatmap(pivot, cmap='YlOrRd', linewidths=0.3, linecolor='white')
    plt.title('年/月 火点数热力图')
    plt.xlabel('月份')
    plt.ylabel('年份')
    plt.tight_layout()
    plt.show()

    # 导出
    ym_out = RESULTS_DIR / 'year_month_fire_counts.csv'
    pivot.to_csv(ym_out)
    print('保存:', ym_out)
else:
    print('无数据，跳过年/月热力图。')

In [ ]:
# 按季节汇总（春夏秋冬）
if len(fire_gdf) > 0:
    season_counts = fire_gdf.groupby('season').size().rename('count').reset_index()
    order = ['Spring','Summer','Autumn','Winter']
    season_counts['season'] = pd.Categorical(season_counts['season'], categories=order, ordered=True)
    season_counts = season_counts.sort_values('season')

    plt.figure(figsize=(6,4))
    sns.barplot(data=season_counts, x='season', y='count', palette='viridis')
    plt.title('按季节汇总的火点数')
    plt.xlabel('季节')
    plt.ylabel('火点数')
    plt.tight_layout()
    plt.show()

    season_out = RESULTS_DIR / 'season_fire_counts.csv'
    season_counts.to_csv(season_out, index=False)
    print('保存:', season_out)
else:
    print('无数据，跳过季节汇总。')

# 趋势分析与显著性检验：思路说明
本节使用非参数 Mann–Kendall 检验与 Sen’s slope 估计：
- 年度总量序列（2010–2019）检验整体趋势及显著性，并输出结果与趋势图。
- 按月份（1–12）构造“逐年同月火点数”序列，分别进行趋势检验，比较各月趋势差异。

In [ ]:
# 年度总量的 Mann–Kendall 趋势检验与 Sen 斜率
import math
from itertools import combinations

if len(fire_gdf) > 0:
    # 选择年份字段
    yr_col = 'year_from_fname' if 'year_from_fname' in fire_gdf.columns else 'year'
    yearly = fire_gdf.groupby(yr_col).size().rename('count').reset_index().sort_values(yr_col)
    years = yearly[yr_col].to_numpy()
    x = years.astype(float)
    y = yearly['count'].to_numpy(dtype=float)

    def mk_test(series):
        s = 0
        n = len(series)
        # S 统计量
        for i in range(n-1):
            diff = series[i+1:] - series[i]
            s += np.sign(diff).sum()
        # 方差（考虑 ties）
        unique, counts = np.unique(series, return_counts=True)
        tie_term = np.sum(counts*(counts-1)*(2*counts+5))
        var_s = (n*(n-1)*(2*n+5) - tie_term) / 18.0
        if s > 0:
            z = (s - 1)/math.sqrt(var_s)
        elif s < 0:
            z = (s + 1)/math.sqrt(var_s)
        else:
            z = 0.0
        # 正态分布双尾 p 值（使用 erf 近似）
        p = 2*(1 - 0.5*(1 + math.erf(abs(z)/math.sqrt(2))))
        return s, var_s, z, p

    def sen_slope(t, series):
        # t 为严格递增的时间（可用年份或索引）
        slopes = []
        n = len(series)
        for i in range(n-1):
            dy = series[i+1:] - series[i]
            dt = t[i+1:] - t[i]
            valid = dt != 0
            if np.any(valid):
                slopes.extend((dy[valid]/dt[valid]).tolist())
        if len(slopes) == 0:
            return 0.0
        return float(np.median(slopes))

    S, VAR_S, Z, P = mk_test(y)
    slope = sen_slope(x, y)
    # Theil–Sen 拦截：median(y - slope*x)
    intercept = float(np.median(y - slope*x))

    # 结果表
    from pandas import DataFrame
    res_df = DataFrame({
        'metric': ['S','VAR_S','Z','p_value','Sen_slope','intercept','n'],
        'value':  [S, VAR_S, Z, P, slope, intercept, len(y)]
    })
    out_csv = RESULTS_DIR / 'yearly_mk_trend_summary.csv'
    res_df.to_csv(out_csv, index=False)
    print('保存:', out_csv)

    # 绘图：年度总量 + 趋势线
    plt.figure(figsize=(8,4))
    sns.lineplot(x=years, y=y, marker='o', label='年度火点数')
    plt.plot(years, intercept + slope*x, color='crimson', linewidth=2, label='趋势线 (Sen)')
    title = f"年度火点数趋势（M-K: Z={Z:.2f}, p={P:.3f}, slope={slope:.1f}/年）"
    plt.title(title)
    plt.xlabel('年份')
    plt.ylabel('火点数')
    plt.legend()
    plt.tight_layout()
    out_png = RESULTS_DIR / 'yearly_trend.png'
    plt.savefig(out_png, dpi=150)
    plt.show()
    print('保存:', out_png)
else:
    print('无数据，跳过年度趋势检验。')

In [ ]:
# 按月逐年序列的 Mann–Kendall 趋势检验
import math

if len(fire_gdf) > 0:
    yr_col = 'year_from_fname' if 'year_from_fname' in fire_gdf.columns else 'year'
    years_all = np.sort(fire_gdf[yr_col].dropna().unique())

    def mk_test(series):
        s = 0
        n = len(series)
        for i in range(n-1):
            diff = series[i+1:] - series[i]
            s += np.sign(diff).sum()
        unique, counts = np.unique(series, return_counts=True)
        tie_term = np.sum(counts*(counts-1)*(2*counts+5))
        var_s = (n*(n-1)*(2*n+5) - tie_term) / 18.0
        if s > 0:
            z = (s - 1)/math.sqrt(var_s)
        elif s < 0:
            z = (s + 1)/math.sqrt(var_s)
        else:
            z = 0.0
        p = 2*(1 - 0.5*(1 + math.erf(abs(z)/math.sqrt(2))))
        return s, var_s, z, p

    def sen_slope(t, series):
        slopes = []
        n = len(series)
        for i in range(n-1):
            dy = series[i+1:] - series[i]
            dt = t[i+1:] - t[i]
            valid = dt != 0
            if np.any(valid):
                slopes.extend((dy[valid]/dt[valid]).tolist())
        if len(slopes) == 0:
            return 0.0
        return float(np.median(slopes))

    rows = []
    for m in range(1, 13):
        sub = fire_gdf[fire_gdf['month'] == m]
        counts = sub.groupby(yr_col).size().reindex(years_all, fill_value=0)
        x = years_all.astype(float)
        y = counts.to_numpy(dtype=float)
        S, VAR_S, Z, P = mk_test(y)
        slope = sen_slope(x, y)
        rows.append({'month': m, 'S': S, 'VAR_S': VAR_S, 'Z': Z, 'p_value': P, 'Sen_slope': slope, 'n': len(y)})

    month_mk = pd.DataFrame(rows)
    out_csv = RESULTS_DIR / 'monthly_mk_trend_summary.csv'
    month_mk.to_csv(out_csv, index=False)
    print('保存:', out_csv)

    # 可视化：按月 Sen 斜率柱状图
    plt.figure(figsize=(9,4))
    sns.barplot(data=month_mk, x='month', y='Sen_slope', palette='crest')
    plt.axhline(0, color='gray', linewidth=1)
    plt.title('逐月 Sen 斜率（年际趋势）')
    plt.xlabel('月份')
    plt.ylabel('Sen 斜率（火点/年）')
    plt.tight_layout()
    out_png = RESULTS_DIR / 'monthly_sen_slopes.png'
    plt.savefig(out_png, dpi=150)
    plt.show()
    print('保存:', out_png)
else:
    print('无数据，跳过逐月趋势检验。')

# 地理分布：思路说明
本节从空间维度刻画火灾活动的主导模式：
- 将火点与县界做空间连接，得到县域尺度的火点统计（总量、分年）。
- 绘制县域层面的连续色斑图（choropleth），展示空间分布格局。
- 输出统计结果到 `results/` 供后续分析或图件制备。

In [ ]:
# 读取县界并进行空间连接
if len(fire_gdf) > 0 and COUNTY_SHP.exists():
    counties = gpd.read_file(COUNTY_SHP)
    # 标准化 CRS 为 WGS84（若非 WGS84，则投影到 EPSG:4326）
    try:
        if counties.crs is None:
            # 常见中国县 shapefile 多为经纬度坐标
            counties = counties.set_crs('EPSG:4326')
        elif counties.crs.to_epsg() != 4326:
            counties = counties.to_crs(4326)
    except Exception:
        counties = counties.to_crs(4326)

    # 空间连接（点在面内）
    join = gpd.sjoin(fire_gdf[['geometry','year']], counties, how='left', predicate='within')

    # 解决 sjoin 造成的同名列冲突（例如 counties 也有 year 字段，会生成 year_left / year_right）
    if 'year_left' in join.columns and 'year_right' in join.columns:
        # 优先使用点表（fire_gdf）的年份信息（year_left）
        join = join.rename(columns={'year_left': 'year'})
        # 去掉面表的 year_right（若需要可改名保留）
        join = join.drop(columns=['year_right'])
    elif 'year_left' in join.columns:
        join = join.rename(columns={'year_left': 'year'})
    elif 'year_right' in join.columns and 'year' not in join.columns:
        join = join.rename(columns={'year_right': 'year'})

    # 尝试找到县级唯一标识字段（若未知，则使用索引）
    candidate_keys = ['county_id','adcode','code','id','County_ID','COUNTY_ID']
    key = None
    for c in candidate_keys:
        if c in join.columns:
            key = c
            break
    if key is None:
        # 若没有明确字段，恢复到 counties 的索引作为标识
        counties = counties.reset_index().rename(columns={'index':'county_idx'})
        join = gpd.sjoin(fire_gdf[['geometry','year']], counties, how='left', predicate='within')
        # 再次处理重命名（同上）
        if 'year_left' in join.columns and 'year_right' in join.columns:
            join = join.rename(columns={'year_left': 'year'}).drop(columns=['year_right'])
        elif 'year_left' in join.columns:
            join = join.rename(columns={'year_left': 'year'})
        elif 'year_right' in join.columns and 'year' not in join.columns:
            join = join.rename(columns={'year_right': 'year'})
        key = 'county_idx'

    # 县域总量
    county_counts = join.groupby(key).size().rename('fire_count').reset_index()
    county_stats = counties.merge(county_counts, on=key, how='left')
    county_stats['fire_count'] = county_stats['fire_count'].fillna(0).astype(int)

    # 县域×年份（确保 join 中有 'year' 列）
    if 'year' not in join.columns:
        raise KeyError("'year' 列在空间连接结果中缺失，无法按年份统计。")
    county_year = join.groupby([key,'year']).size().rename('fire_count').reset_index()

    # 导出
    out_county_all = RESULTS_DIR / 'county_fire_counts.csv'
    out_county_year = RESULTS_DIR / 'county_year_fire_counts.csv'
    county_stats.drop(columns='geometry').to_csv(out_county_all, index=False)
    county_year.to_csv(out_county_year, index=False)
    print('保存:', out_county_all)
    print('保存:', out_county_year)

    # 绘制县域总量分布
    plt.figure(figsize=(8,8))
    ax = counties.boundary.plot(edgecolor='lightgray', linewidth=0.3)
    county_stats.plot(column='fire_count', ax=ax, legend=True, cmap='OrRd', linewidth=0, alpha=0.9)
    plt.title('县域火点总量空间分布（2010–2019 合并）')
    plt.axis('off')
    plt.tight_layout()
    plt.show()
else:
    if not COUNTY_SHP.exists():
        print('未找到县界文件:', COUNTY_SHP)
    else:
        print('无火点数据，跳过空间连接。')

# 周尺度与季节内模式：思路说明
本节旨在超越“月度统计”，识别更细粒度的时间结构：
- 周尺度（ISO 周）：
  - 生成年×周热力图，观察热点周是否跨年一致；
  - 计算跨年每周的均值/中位数/分位数，找出“热点周”。
- 季节内（week-in-season）：
  - 将每季按周序列化，比较季节内部的固定峰周或变动周；
  - 识别各季的热点周并导出结果。

In [ ]:
# 周尺度聚合与热点周识别（ISO Week）
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

if len(fire_gdf) > 0:
    # 计算 ISO 年/周（注意：返回的是 Int64 类型，需要转 int）
    iso = fire_gdf['datetime'].dt.isocalendar()
    fire_gdf['iso_year'] = iso['year'].astype(int)
    fire_gdf['iso_week'] = iso['week'].astype(int)

    weekly = fire_gdf.groupby(['iso_year','iso_week']).size().rename('count').reset_index()

    # 年×周热力图
    years = np.sort(weekly['iso_year'].unique())
    weeks = np.arange(1, 54)  # 1..53 周
    pivot = weekly.pivot(index='iso_year', columns='iso_week', values='count')
    pivot = pivot.reindex(index=years, columns=weeks, fill_value=0)

    plt.figure(figsize=(12, 6))
    sns.heatmap(pivot, cmap='YlOrRd', linewidths=0.3, linecolor='white')
    plt.title('年 × ISO周 火点数热力图')
    plt.xlabel('ISO Week')
    plt.ylabel('ISO Year')
    plt.tight_layout()
    heat_out = RESULTS_DIR / 'year_week_fire_counts_heatmap.png'
    plt.savefig(heat_out, dpi=150)
    plt.show()
    print('保存:', heat_out)

    # 导出年×周表
    yw_out = RESULTS_DIR / 'year_week_fire_counts.csv'
    pivot.to_csv(yw_out)
    print('保存:', yw_out)

    # 跨年每周统计（均值/中位数/分位）
    stats = pd.DataFrame({
        'week': weeks,
        'mean': pivot.mean(axis=0).values,
        'median': pivot.median(axis=0).values,
        'p10': pivot.quantile(0.10, axis=0).values,
        'p90': pivot.quantile(0.90, axis=0).values
    })
    stats_out = RESULTS_DIR / 'week_of_year_stats.csv'
    stats.to_csv(stats_out, index=False)
    print('保存:', stats_out)

    # 识别热点周（两种方式：Top-N 与阈值）
    top_n = 10
    top_weeks = stats.sort_values('mean', ascending=False).head(top_n).copy()
    # 阈值：均值 >= 均值均值 + 1 标准差
    mu, sigma = stats['mean'].mean(), stats['mean'].std(ddof=0)
    thr = mu + sigma
    high_weeks = stats[stats['mean'] >= thr].copy()
    hot = top_weeks.merge(high_weeks, on='week', how='outer', suffixes=('_top',''))
    hot = hot.sort_values('mean', ascending=False).reset_index(drop=True)
    hot_out = RESULTS_DIR / 'hot_weeks_summary.csv'
    hot.to_csv(hot_out, index=False)
    print('保存:', hot_out)

    # 可视化：每周均值曲线 + p10-p90 区间
    plt.figure(figsize=(10,4))
    plt.fill_between(stats['week'], stats['p10'], stats['p90'], color='#FDE725', alpha=0.25, label='P10–P90')
    plt.plot(stats['week'], stats['mean'], color='#2C7FB8', linewidth=2, label='Mean')
    plt.title('跨年每周火点数统计（均值与分位区间）')
    plt.xlabel('ISO Week')
    plt.ylabel('火点数')
    plt.legend()
    plt.tight_layout()
    wcurve_out = RESULTS_DIR / 'week_of_year_curve.png'
    plt.savefig(wcurve_out, dpi=150)
    plt.show()
    print('保存:', wcurve_out)
else:
    print('无数据，跳过周尺度分析。')

In [ ]:
# 季节内（week-in-season）模式与热点周
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

if len(fire_gdf) > 0:
    # 计算 season_year：Winter 跨年，12月属于当年冬季，其余 1-2 月属于上一年冬季
    def compute_season_year(row):
        m = row['month']
        y = row['year']
        s = row['season']
        if s == 'Winter':
            if m == 12:
                return y
            else:  # 1,2 月
                return y - 1
        else:
            return y

    fire_gdf['season_year'] = fire_gdf.apply(compute_season_year, axis=1)

    # 计算季节起始日期
    season_start_month = {'Spring':3,'Summer':6,'Autumn':9,'Winter':12}
    def compute_season_start(row):
        sy = int(row['season_year'])
        mon = season_start_month.get(row['season'], 1)
        return datetime(sy, mon, 1)

    fire_gdf['season_start'] = fire_gdf.apply(compute_season_start, axis=1)

    # 计算季节内的周序号（从1开始）
    delta_days = (fire_gdf['datetime'] - fire_gdf['season_start']).dt.days
    fire_gdf['week_in_season'] = (delta_days // 7).astype(int) + 1

    # 聚合：按 season_year × season × week_in_season 统计
    ssw = fire_gdf.groupby(['season_year','season','week_in_season']).size().rename('count').reset_index()

    # 跨年统计：对每个 season 的同一 week_in_season 计算均值/分位
    def season_stats(season_name):
        sub = ssw[ssw['season'] == season_name]
        if sub.empty:
            return pd.DataFrame(columns=['season','week_in_season','mean','median','p10','p90'])
        weeks = np.sort(sub['week_in_season'].unique())
        # 构造 pivot: 行=season_year，列=week_in_season
        pivot = sub.pivot(index='season_year', columns='week_in_season', values='count').reindex(columns=weeks).fillna(0)
        return pd.DataFrame({
            'season': season_name,
            'week_in_season': weeks,
            'mean': pivot.mean(axis=0).values,
            'median': pivot.median(axis=0).values,
            'p10': pivot.quantile(0.10, axis=0).values,
            'p90': pivot.quantile(0.90, axis=0).values
        })

    seasons = ['Spring','Summer','Autumn','Winter']
    stats_list = [season_stats(s) for s in seasons]
    season_stats_df = pd.concat(stats_list, ignore_index=True) if stats_list else pd.DataFrame()

    # 导出季节内统计
    sstats_out = RESULTS_DIR / 'season_week_stats.csv'
    season_stats_df.to_csv(sstats_out, index=False)
    print('保存:', sstats_out)

    # 识别各季热点周（按均值 Top-3 + 阈值）
    hot_rows = []
    for s in seasons:
        sub = season_stats_df[season_stats_df['season'] == s]
        if sub.empty:
            continue
        top3 = sub.sort_values('mean', ascending=False).head(3).copy()
        mu, sigma = sub['mean'].mean(), sub['mean'].std(ddof=0)
        thr = mu + sigma
        over = sub[sub['mean'] >= thr].copy()
        top3['flag'] = 'top3'
        over['flag'] = '>=mean+1sd'
        hot_rows.append(pd.concat([top3, over]).drop_duplicates(subset=['week_in_season']))
    season_hot = pd.concat(hot_rows, ignore_index=True) if hot_rows else pd.DataFrame()
    shot_out = RESULTS_DIR / 'season_hotspot_weeks.csv'
    season_hot.to_csv(shot_out, index=False)
    print('保存:', shot_out)

    # 绘图：四季子图，均值线 + P10-P90 阴影
    fig, axes = plt.subplots(2, 2, figsize=(12, 8), sharex=False, sharey=False)
    axes = axes.ravel()
    for i, s in enumerate(seasons):
        ax = axes[i]
        sub = season_stats_df[season_stats_df['season'] == s]
        if sub.empty:
            ax.set_title(f'{s}（无数据）')
            ax.axis('off')
            continue
        ax.fill_between(sub['week_in_season'], sub['p10'], sub['p90'], color='#FDE725', alpha=0.25, label='P10–P90')
        ax.plot(sub['week_in_season'], sub['mean'], color='#2C7FB8', linewidth=2, label='Mean')
        ax.set_title(f'{s} 季节内周序列')
        ax.set_xlabel('季节内周序号')
        ax.set_ylabel('火点数')
        ax.legend(fontsize=8)
    plt.tight_layout()
    sw_plot = RESULTS_DIR / 'season_week_curves.png'
    plt.savefig(sw_plot, dpi=150)
    plt.show()
    print('保存:', sw_plot)
else:
    print('无数据，跳过季节内周模式分析。')

In [ ]:
# 自动生成简短摘要（可选）
# 依据“热点周”与“季节内热点周”生成要点摘要，便于写作引用
try:
    lines = []
    # 周尺度热点
    hot_path = RESULTS_DIR / 'hot_weeks_summary.csv'
    if hot_path.exists():
        hot = pd.read_csv(hot_path)
        if not hot.empty:
            top_rows = hot.sort_values('mean', ascending=False).head(5)
            weeks_str = ', '.join(str(int(w)) for w in top_rows['week'].tolist())
            lines.append(f"历史上均值最高的热点周（Top-5）：{weeks_str}（单位：周序号）")
    # 季节内热点
    shot_path = RESULTS_DIR / 'season_hotspot_weeks.csv'
    if shot_path.exists():
        shot = pd.read_csv(shot_path)
        if not shot.empty:
            for s in ['Spring','Summer','Autumn','Winter']:
                sub = shot[shot['season']==s].sort_values('mean', ascending=False)
                top = sub.head(3)
                if not top.empty:
                    wks = ', '.join(str(int(w)) for w in top['week_in_season'].tolist())
                    lines.append(f"{s}热点周（Top-3）：{wks}")
    if lines:
        summary_path = RESULTS_DIR / 'summary_insights.txt'
        with open(summary_path, 'w', encoding='utf-8') as f:
            f.write('\n'.join(lines))
        print('保存:', summary_path)
    else:
        print('暂无可写摘要（可能数据为空或前序单元未运行）。')
except Exception as e:
    print('生成摘要失败:', e)

# 热点周的空间分布：思路说明
为回答“热点周落在哪些区域更集中”，本节将：
- 读取已识别的热点周（基于跨年周均值 Top-N 与阈值筛选）。
- 以这些周的火点为子集，与县界做空间连接，统计“县域总量（热点周期间）”。
- 绘制：
  1) 聚合所有热点周的县域色斑图；
  2) 前 Top-K 热点周分别绘制县域色斑图，观察是否空间位置稳定。

In [ ]:
# 热点周（Top-K）的县域空间分布
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

TOP_K_WEEKS = 3  # 可调：绘制前 K 个热点周

if len(fire_gdf) > 0 and COUNTY_SHP.exists():
    # 读热点周列表
    hot_path = RESULTS_DIR / 'hot_weeks_summary.csv'
    if not hot_path.exists():
        print('未找到热点周文件，尝试使用 week_of_year_stats.csv 的 Top 周替代。')
        stats_path = RESULTS_DIR / 'week_of_year_stats.csv'
        if stats_path.exists():
            stats = pd.read_csv(stats_path)
            hot_weeks = stats.sort_values('mean', ascending=False).head(TOP_K_WEEKS)['week'].astype(int).tolist()
        else:
            print('week_of_year_stats.csv 也不存在，无法进行热点周空间分析。')
            hot_weeks = []
    else:
        hot = pd.read_csv(hot_path)
        if 'week' not in hot.columns:
            print('hot_weeks_summary.csv 缺少 week 列，无法继续。')
            hot_weeks = []
        else:
            hot_weeks = hot.dropna(subset=['week']).sort_values('mean', ascending=False)['week'].astype(int).unique().tolist()
            hot_weeks = hot_weeks[:TOP_K_WEEKS]

    if len(hot_weeks) == 0:
        print('没有可用的热点周。')
    else:
        print('选定热点周（Top-K）:', hot_weeks)
        # 确保已计算 iso_week
        if 'iso_week' not in fire_gdf.columns:
            iso = fire_gdf['datetime'].dt.isocalendar()
            fire_gdf['iso_year'] = iso['year'].astype(int)
            fire_gdf['iso_week'] = iso['week'].astype(int)

        # 读取县界
        counties = gpd.read_file(COUNTY_SHP)
        try:
            if counties.crs is None:
                counties = counties.set_crs('EPSG:4326')
            elif counties.crs.to_epsg() != 4326:
                counties = counties.to_crs(4326)
        except Exception:
            counties = counties.to_crs(4326)

        # 寻找县级标识键
        candidate_keys = ['county_id','adcode','code','id','County_ID','COUNTY_ID']
        def detect_key(df_cols):
            for c in candidate_keys:
                if c in df_cols:
                    return c
            return None

        # 1) 聚合所有热点周的总量
        sub_all = fire_gdf[fire_gdf['iso_week'].isin(hot_weeks)][['geometry','year']].copy()
        join_all = gpd.sjoin(sub_all, counties, how='left', predicate='within')
        # 处理 year 列冲突
        if 'year_left' in join_all.columns and 'year_right' in join_all.columns:
            join_all = join_all.rename(columns={'year_left':'year'}).drop(columns=['year_right'])
        elif 'year_left' in join_all.columns:
            join_all = join_all.rename(columns={'year_left':'year'})
        elif 'year_right' in join_all.columns and 'year' not in join_all.columns:
            join_all = join_all.rename(columns={'year_right':'year'})

        key = detect_key(join_all.columns)
        if key is None:
            counties = counties.reset_index().rename(columns={'index':'county_idx'})
            join_all = gpd.sjoin(sub_all, counties, how='left', predicate='within')
            if 'year_left' in join_all.columns and 'year_right' in join_all.columns:
                join_all = join_all.rename(columns={'year_left':'year'}).drop(columns=['year_right'])
            elif 'year_left' in join_all.columns:
                join_all = join_all.rename(columns={'year_left':'year'})
            elif 'year_right' in join_all.columns and 'year' not in join_all.columns:
                join_all = join_all.rename(columns={'year_right':'year'})
            key = 'county_idx'

        county_counts_all = join_all.groupby(key).size().rename('fire_count').reset_index()
        county_stats_all = counties.merge(county_counts_all, on=key, how='left')
        county_stats_all['fire_count'] = county_stats_all['fire_count'].fillna(0).astype(int)

        out_csv_all = RESULTS_DIR / 'county_hot_weeks_fire_counts.csv'
        county_stats_all.drop(columns='geometry').to_csv(out_csv_all, index=False)
        print('保存:', out_csv_all)

        plt.figure(figsize=(8,8))
        ax = counties.boundary.plot(edgecolor='lightgray', linewidth=0.3)
        county_stats_all.plot(column='fire_count', ax=ax, legend=True, cmap='OrRd', linewidth=0, alpha=0.9)
        plt.title(f'县域火点空间分布（聚合热点周：{hot_weeks}）')
        plt.axis('off')
        plt.tight_layout()
        out_png_all = RESULTS_DIR / 'county_hot_weeks_map.png'
        plt.savefig(out_png_all, dpi=150)
        plt.show()
        print('保存:', out_png_all)

        # 2) 分别绘制前 TOP_K 周的地图
        for w in hot_weeks:
            sub_w = fire_gdf[fire_gdf['iso_week']==w][['geometry','year']].copy()
            join_w = gpd.sjoin(sub_w, counties, how='left', predicate='within')
            if 'year_left' in join_w.columns and 'year_right' in join_w.columns:
                join_w = join_w.rename(columns={'year_left':'year'}).drop(columns=['year_right'])
            elif 'year_left' in join_w.columns:
                join_w = join_w.rename(columns={'year_left':'year'})
            elif 'year_right' in join_w.columns and 'year' not in join_w.columns:
                join_w = join_w.rename(columns={'year_right':'year'})

            key_w = detect_key(join_w.columns)
            if key_w is None:
                key_w = key  # 回退到上面选择的键

            county_counts_w = join_w.groupby(key_w).size().rename('fire_count').reset_index()
            county_stats_w = counties.merge(county_counts_w, on=key_w, how='left')
            county_stats_w['fire_count'] = county_stats_w['fire_count'].fillna(0).astype(int)

            out_csv_w = RESULTS_DIR / f'county_hot_week_{int(w)}_fire_counts.csv'
            county_stats_w.drop(columns='geometry').to_csv(out_csv_w, index=False)
            print('保存:', out_csv_w)

            plt.figure(figsize=(8,8))
            ax = counties.boundary.plot(edgecolor='lightgray', linewidth=0.3)
            county_stats_w.plot(column='fire_count', ax=ax, legend=True, cmap='OrRd', linewidth=0, alpha=0.9)
            plt.title(f'县域火点空间分布（热点周：ISO Week {int(w)}）')
            plt.axis('off')
            plt.tight_layout()
            out_png_w = RESULTS_DIR / f'county_hot_week_{int(w)}_map.png'
            plt.savefig(out_png_w, dpi=150)
            plt.show()
            print('保存:', out_png_w)
else:
    if not COUNTY_SHP.exists():
        print('未找到县界文件:', COUNTY_SHP)
    else:
        print('无火点数据，跳过热点周空间分析。')

In [ ]:
# 季节分组的县域空间分布（可选增强）
# 将空间分布与季节性关联：按四季分别绘制县域色斑图，便于比较季节差异。
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

if len(fire_gdf) > 0 and COUNTY_SHP.exists():
    counties = gpd.read_file(COUNTY_SHP)
    try:
        if counties.crs is None:
            counties = counties.set_crs('EPSG:4326')
        elif counties.crs.to_epsg() != 4326:
            counties = counties.to_crs(4326)
    except Exception:
        counties = counties.to_crs(4326)

    seasons = ['Spring','Summer','Autumn','Winter']

    # 寻找县级标识键
    candidate_keys = ['county_id','adcode','code','id','County_ID','COUNTY_ID']
    def detect_key(df_cols):
        for c in candidate_keys:
            if c in df_cols:
                return c
        return None

    all_rows = []
    for s in seasons:
        sub = fire_gdf[fire_gdf['season']==s][['geometry','year']].copy()
        if sub.empty:
            continue
        join = gpd.sjoin(sub, counties, how='left', predicate='within')
        if 'year_left' in join.columns and 'year_right' in join.columns:
            join = join.rename(columns={'year_left':'year'}).drop(columns=['year_right'])
        elif 'year_left' in join.columns:
            join = join.rename(columns={'year_left':'year'})
        elif 'year_right' in join.columns and 'year' not in join.columns:
            join = join.rename(columns={'year_right':'year'})

        key = detect_key(join.columns)
        if key is None:
            counties = counties.reset_index().rename(columns={'index':'county_idx'})
            join = gpd.sjoin(sub, counties, how='left', predicate='within')
            if 'year_left' in join.columns and 'year_right' in join.columns:
                join = join.rename(columns={'year_left':'year'}).drop(columns=['year_right'])
            elif 'year_left' in join.columns:
                join = join.rename(columns={'year_left':'year'})
            elif 'year_right' in join.columns and 'year' not in join.columns:
                join = join.rename(columns={'year_right':'year'})
            key = 'county_idx'

        # 县域总量（该季汇总）
        counts = join.groupby(key).size().rename('fire_count').reset_index()
        counts['season'] = s
        all_rows.append(counts)

        stats_s = counties.merge(counts, on=key, how='left')
        stats_s['fire_count'] = stats_s['fire_count'].fillna(0).astype(int)

        out_csv_s = RESULTS_DIR / f'county_season_{s}_fire_counts.csv'
        stats_s.drop(columns='geometry').to_csv(out_csv_s, index=False)
        print('保存:', out_csv_s)

        plt.figure(figsize=(8,8))
        ax = counties.boundary.plot(edgecolor='lightgray', linewidth=0.3)
        stats_s.plot(column='fire_count', ax=ax, legend=True, cmap='OrRd', linewidth=0, alpha=0.9)
        plt.title(f'县域火点空间分布（{s}）')
        plt.axis('off')
        plt.tight_layout()
        out_png_s = RESULTS_DIR / f'county_season_{s}_map.png'
        plt.savefig(out_png_s, dpi=150)
        plt.show()
        print('保存:', out_png_s)

    if all_rows:
        all_df = pd.concat(all_rows, ignore_index=True)
        out_all = RESULTS_DIR / 'county_season_fire_counts.csv'
        all_df.to_csv(out_all, index=False)
        print('保存:', out_all)
else:
    if not COUNTY_SHP.exists():
        print('未找到县界文件:', COUNTY_SHP)
    else:
        print('无火点数据，跳过季节空间分析。')

# 核心任务 2：农业焚烧归因（玉米与小麦收获后）
本节目标：
- 方法学：基于“空间位置相对于耕地 + 时间位置相对于收获季”的双阈组合判定。
- 产出：
  1) “可能为农业焚烧”的火点比例（%）；
  2) 分类点图与县域尺度分类色斑图；
  3) 可复用的分类表（CSV）。

数据期望：
- cropland（耕地）：优先使用 `raw_data/cropland_distribution_and_phenological_data/` 下的矢量（*.shp）或栅格（*.tif）。
- harvest calendar（可选）：若目录下有 `harvest_calendar.csv`，优先按区域自定义时段，否则使用默认全国窗口。

In [ ]:
# 农业焚烧分类与导出（点级 + 县域汇总 + 地图）
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [ ]:
# 自动耕地标记（优先矢量，其次栅格；标记 fire_gdf['is_cropland']）
import warnings
from pathlib import Path
import geopandas as gpd
import pandas as pd

try:
    from shapely.strtree import STRtree
    _HAS_STRTREE = True
except Exception:
    _HAS_STRTREE = False

try:
    from shapely.prepared import prep
except Exception:
    prep = None


def _fallback_point_within(points_gdf: gpd.GeoDataFrame, polygons_gdf: gpd.GeoDataFrame) -> pd.DataFrame:
    """降级实现 point-in-polygon，用于缺少空间索引时。"""
    if len(points_gdf) == 0 or len(polygons_gdf) == 0:
        result = points_gdf[['index', 'geometry']].copy()
        result['index_right'] = [None] * len(result)
        return result

    result = points_gdf[['index', 'geometry']].copy()
    poly_geoms = list(polygons_gdf.geometry)
    poly_indices = list(polygons_gdf.index)
    matches = []

    if _HAS_STRTREE:
        tree = STRtree(poly_geoms)
        geom_map = {geom.wkb: poly_indices[idx] for idx, geom in enumerate(poly_geoms)}
        for geom in result.geometry:
            found = None
            for candidate in tree.query(geom):
                if candidate.contains(geom) or candidate.covers(geom):
                    found = geom_map.get(candidate.wkb)
                    if found is None:
                        found = next((poly_indices[i] for i, g in enumerate(poly_geoms) if g.equals(candidate)), None)
                    break
            matches.append(found)
    else:
        prepared = [(prep(poly), idx) for poly, idx in zip(poly_geoms, poly_indices)] if prep is not None else []
        for geom in result.geometry:
            found = None
            for prepared_poly, idx in prepared:
                if prepared_poly.contains(geom) or prepared_poly.covers(geom):
                    found = idx
                    break
            if found is None:
                for idx, poly in zip(poly_indices, poly_geoms):
                    if poly.contains(geom) or poly.covers(geom):
                        found = idx
                        break
            matches.append(found)

    result['index_right'] = matches
    return result


def _geometry_to_xy(geom):
    """为点/多点/面几何提取一个代表性的 (x, y) 坐标，用于栅格采样。"""
    if geom is None or geom.is_empty:
        return None
    gtype = geom.geom_type
    if gtype == 'Point':
        return geom.x, geom.y
    if gtype in {'MultiPoint', 'GeometryCollection'}:
        for part in geom.geoms:
            xy = _geometry_to_xy(part)
            if xy is not None:
                return xy
        return None
    centroid = geom.centroid
    if centroid.is_empty:
        return None
    return centroid.x, centroid.y


# 可选：强制重算 is_cropland（用于修正之前阈值/越界带来的异常）
FORCE_RECOMPUTE_IS_CROPLAND = True

CROP_DIR = REPO_ROOT / 'raw_data' / 'cropland_distribution_and_phenological_data'

if 'fire_gdf' not in globals() or fire_gdf is None or len(fire_gdf) == 0:
    raise RuntimeError('fire_gdf 缺失，请先运行数据加载或分类单元中的兜底装载。')

# 若已有有效 is_cropland，且不强制重算，则跳过
if (not FORCE_RECOMPUTE_IS_CROPLAND) and ('is_cropland' in fire_gdf.columns) and (fire_gdf['is_cropland'].sum() > 0):
    print('检测到已有有效 is_cropland，不再重复标记。')
else:
    # 如存在旧列，先清空
    if 'is_cropland' in fire_gdf.columns:
        del fire_gdf['is_cropland']

    shp_candidates = list(CROP_DIR.rglob('*.shp')) if CROP_DIR.exists() else []
    tif_candidates = list(CROP_DIR.rglob('*.tif')) if CROP_DIR.exists() else []

    marked = False

    # 1) 矢量耕地优先：点落在多边形内即视为耕地
    if shp_candidates:
        try:
            crop_shp = max(shp_candidates, key=lambda p: p.stat().st_size)
            print('使用耕地矢量：', crop_shp)
            cropland_gdf = gpd.read_file(crop_shp)
            if cropland_gdf.crs is None:
                cropland_gdf = cropland_gdf.set_crs('EPSG:4326')
            elif cropland_gdf.crs.to_epsg() != 4326:
                cropland_gdf = cropland_gdf.to_crs(4326)

            left = gpd.GeoDataFrame(
                fire_gdf[['geometry']].reset_index(),
                geometry='geometry',
                crs=fire_gdf.crs
            )
            right = gpd.GeoDataFrame(
                cropland_gdf[['geometry']],
                geometry='geometry',
                crs=cropland_gdf.crs
            )

            try:
                join = gpd.sjoin(left, right, how='left', predicate='within')
            except Exception as exc:
                if 'Spatial indexes require' in str(exc):
                    warnings.warn('未检测到 rtree/pygeos，使用 Shapely 兜底空间关联，速度可能较慢。')
                    join = _fallback_point_within(left, right)
                else:
                    raise

            is_crop = join['index_right'].notna().values
            fire_gdf['is_cropland'] = is_crop
            print('矢量判定完成：is_cropland=True 计数 =', int(fire_gdf['is_cropland'].sum()))
            marked = True
        except Exception as e:
            print('矢量耕地判定失败：', e)

    # 2) 栅格耕地：采样栅格值，考虑 nodata 与范围，使用自适应阈值
    if not marked and tif_candidates:
        try:
            import numpy as np
            import rasterio

            raster_override = globals().get('cropland_raster_path')
            if raster_override:
                raw_candidates = [Path(raster_override)]
            else:
                raw_candidates = tif_candidates

            raster_files = []
            for candidate in raw_candidates:
                candidate = Path(candidate)
                if candidate.is_dir():
                    raster_files.extend(sorted(candidate.rglob('*.tif')))
                elif candidate.suffix.lower() == '.tif' and candidate.exists():
                    raster_files.append(candidate)

            if not raster_files:
                raise FileNotFoundError('未找到可用的耕地栅格 (.tif) 文件，请确认路径。')

            rpath = max(raster_files, key=lambda p: p.stat().st_size).resolve()
            print('使用耕地栅格：', rpath)

            with rasterio.open(rpath) as src:
                xy_coords = [_geometry_to_xy(geom) for geom in fire_gdf.geometry]
                valid_pairs = [(idx, xy) for idx, xy in enumerate(xy_coords) if xy is not None]
                arr = np.full(len(xy_coords), np.nan, dtype='float32')

                if valid_pairs:
                    bounds = src.bounds
                    inside = []
                    for idx, (x, y) in valid_pairs:
                        in_bounds = (bounds.left <= x <= bounds.right) and (bounds.bottom <= y <= bounds.top)
                        if in_bounds:
                            inside.append((idx, (x, y)))

                    if inside:
                        sample_pts = [pt for _, pt in inside]
                        sample_idx = [idx for idx, _ in inside]
                        samp = np.array(list(src.sample(sample_pts))).squeeze()
                        if samp.ndim == 0:
                            samp = np.array([samp], dtype='float32')
                        else:
                            samp = samp.astype('float32', copy=False)

                        ndv = src.nodata
                        if ndv is not None and np.isfinite(ndv):
                            samp = np.where(np.isclose(samp, ndv), np.nan, samp)

                        arr[sample_idx] = samp

                finite = np.isfinite(arr)
                if finite.any():
                    vmax = float(np.nanmax(arr))
                    if vmax <= 1.0:
                        thr = 0.5
                        rule = '> 0.5'
                    elif vmax <= 100.0:
                        thr = 50.0
                        rule = '>= 50%'
                    else:
                        thr = 0.0
                        rule = '> 0'
                    is_crop = (finite & (arr > thr))
                else:
                    is_crop = np.zeros(len(arr), dtype=bool)
                    thr = None
                    rule = 'no-data'

                fire_gdf['is_cropland'] = is_crop
                true_cnt = int(is_crop.sum())
                print(f'栅格判定完成（阈值规则 {rule}）：is_cropland=True 计数 = {true_cnt}')
                marked = True
        except ImportError:
            print('需要 rasterio 来采样栅格，请先安装：pip install rasterio（或在 Notebook 内安装）。')
        except Exception as e:
            print('栅格耕地判定失败：', e)

    if not marked:
        if 'is_cropland' not in fire_gdf.columns:
            fire_gdf['is_cropland'] = False
        print('未能完成耕地标记，已兜底 is_cropland=False（请检查 cropland 数据与依赖）。')

# 简要汇总
total = len(fire_gdf)
true_cnt = int(fire_gdf['is_cropland'].sum()) if 'is_cropland' in fire_gdf.columns else 0
print(f'is_cropland 汇总：True={true_cnt} / Total={total} ({true_cnt/total*100 if total else 0:.2f}%)')


In [ ]:
# 农业焚烧分类与导出（点级 + 县域汇总 + 地图）——稳健版（含兜底加载与缺失列补全）
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from shapely.geometry import Point

# 统一中文字体，抑制中文缺字警告
try:
    from matplotlib import font_manager, rcParams
    preferred = ['Microsoft YaHei','SimHei','Source Han Sans SC','Noto Sans CJK SC']
    available = {f.name for f in font_manager.fontManager.ttflist}
    hit = None
    for name in preferred:
        if name in available:
            hit = name
            break
    if hit:
        rcParams['font.sans-serif'] = [hit] + rcParams.get('font.sans-serif', [])
        rcParams['axes.unicode_minus'] = False
except Exception:
    pass

# 0) 路径与目录兜底
if 'REPO_ROOT' not in globals() or 'RESULTS_DIR' not in globals() or 'RAW_DIR' not in globals() or 'FIRE_DIR' not in globals() or 'COUNTY_SHP' not in globals():
    def detect_repo_root(start: Path = Path.cwd()) -> Path:
        for p in [start] + list(start.parents):
            if (p / 'raw_data').exists():
                return p
        return start.parent if start.name == 'script' else start
    REPO_ROOT = detect_repo_root()
    RAW_DIR = REPO_ROOT / 'raw_data'
    FIRE_DIR = RAW_DIR / 'satellite_fire_data'
    COUNTY_SHP = RAW_DIR / 'chn_county' / 'chn_county.shp'
    RESULTS_DIR = REPO_ROOT / 'results'
else:
    # 确保所有路径变量都是 Path 对象
    if not isinstance(RESULTS_DIR, Path):
        RESULTS_DIR = Path(RESULTS_DIR)
    if not isinstance(FIRE_DIR, Path):
        FIRE_DIR = Path(FIRE_DIR)
    if not isinstance(RAW_DIR, Path):
        RAW_DIR = Path(RAW_DIR)
    if not isinstance(REPO_ROOT, Path):
        REPO_ROOT = Path(REPO_ROOT)
    if not isinstance(COUNTY_SHP, Path):
        COUNTY_SHP = Path(COUNTY_SHP)

RESULTS_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_TASK2 = RESULTS_DIR / 'task2' 
RESULTS_TASK2.mkdir(parents=True, exist_ok=True)

# 1) 基础工具兜底
if 'find_column' not in globals():
    def find_column(cols, candidates):
        for cand in candidates:
            for col in cols:
                if col.lower() == cand.lower():
                    return col
        return None

if 'build_datetime' not in globals():
    def build_datetime(df):
        date_col = find_column(df.columns, ['acq_date', 'date', 'acq-date'])
        time_col = find_column(df.columns, ['acq_time', 'time', 'acq-time'])
        dt = None
        if date_col is not None and time_col is not None:
            def _fmt_time(t):
                try:
                    s = str(int(t)).zfill(4)
                    return f"{s[:2]}:{s[2:]}"
                except Exception:
                    return None
            times = df[time_col].apply(_fmt_time)
            dt = pd.to_datetime(df[date_col].astype(str) + ' ' + times.astype(str), errors='coerce')
        elif date_col is not None:
            dt = pd.to_datetime(df[date_col], errors='coerce')
        else:
            ts_col = find_column(df.columns, ['timestamp'])
            if ts_col is not None:
                dt = pd.to_datetime(df[ts_col], errors='coerce')
        if dt is None:
            raise ValueError('未能识别日期/时间列，请检查原始 CSV 列名。')
        df['datetime'] = dt
        df['date'] = df['datetime'].dt.date
        df['year'] = df['datetime'].dt.year
        df['month'] = df['datetime'].dt.month
        df['day'] = df['datetime'].dt.day
        MONTH2SEASON = {12:'Winter',1:'Winter',2:'Winter',3:'Spring',4:'Spring',5:'Spring',6:'Summer',7:'Summer',8:'Summer',9:'Autumn',10:'Autumn',11:'Autumn'}
        df['season'] = df['month'].map(MONTH2SEASON)
        return df

if 'to_geodf' not in globals():
    def to_geodf(df):
        lon_col = find_column(df.columns, ['longitude', 'lon', 'long'])
        lat_col = find_column(df.columns, ['latitude', 'lat'])
        if lon_col is None or lat_col is None:
            raise ValueError('未找到经纬度列（longitude/latitude 或 lon/lat）。')
        geometry = [Point(xy) for xy in zip(df[lon_col].astype(float), df[lat_col].astype(float))]
        gdf = gpd.GeoDataFrame(df.copy(), geometry=geometry, crs='EPSG:4326')
        return gdf

# 2) 数据兜底：若 fire_gdf 缺失则自动装载
if 'fire_gdf' not in globals() or fire_gdf is None or len(fire_gdf) == 0:
    from glob import glob
    # 确保 FIRE_DIR 是 Path 对象
    if not isinstance(FIRE_DIR, Path):
        FIRE_DIR = Path(FIRE_DIR)
    
    csv_paths = sorted(FIRE_DIR.glob('modis_*_china.csv'))
    csv_paths = [str(p) for p in csv_paths]  # 转为字符串列表以兼容 pd.read_csv
    frames = []

    for p in csv_paths:
        try:
            df = pd.read_csv(p)
            df = build_datetime(df)
            lon_col = find_column(df.columns, ['longitude', 'lon', 'long'])
            lat_col = find_column(df.columns, ['latitude', 'lat'])
            if lon_col and lat_col:
                # 粗略范围裁剪到中国，避免异常点
                df = df[(df[lon_col].between(70, 140)) & (df[lat_col].between(15, 55))]
            frames.append(df)
        except Exception as e:
            print(f'读取失败 {p}: {e}')
    fire_df = pd.DataFrame() if not frames else pd.concat(frames, ignore_index=True)
    fire_gdf = to_geodf(fire_df).set_crs('EPSG:4326', allow_override=True) if not fire_df.empty else gpd.GeoDataFrame(columns=['datetime','year','month','season','geometry'], geometry='geometry', crs='EPSG:4326')
print('fire_gdf 就绪，记录数:', len(fire_gdf))

# 3) 时间窗口标记兜底：若未运行专门单元则按全国默认窗口生成
if 'in_wheat_window' not in fire_gdf.columns or 'in_corn_window' not in fire_gdf.columns:
    from datetime import datetime
    DEFAULT_WINDOWS = [
        {'crop':'wheat', 'start':(5,15), 'end':(6,30)},   # 冬小麦
        {'crop':'wheat', 'start':(8,1),  'end':(9,15)},   # 北方春小麦/补充
        {'crop':'corn',  'start':(9,1),  'end':(11,15)},  # 玉米
    ]
    def in_mmdd_window(dt: pd.Timestamp, sm, sd, em, ed):
        if pd.isna(dt):
            return False
        m, d = dt.month, dt.day
        if (sm,sd) <= (em,ed):
            return (m,d) >= (sm,sd) and (m,d) <= (em,ed)
        else:
            return (m,d) >= (sm,sd) or (m,d) <= (em,ed)
    fire_gdf['in_wheat_window'] = False
    fire_gdf['in_corn_window'] = False
    for win in DEFAULT_WINDOWS:
        flag = fire_gdf['datetime'].apply(lambda t: in_mmdd_window(t, win['start'][0], win['start'][1], win['end'][0], win['end'][1]))
        if win['crop'] == 'wheat':
            fire_gdf['in_wheat_window'] = fire_gdf['in_wheat_window'] | flag
        elif win['crop'] == 'corn':
            fire_gdf['in_corn_window'] = fire_gdf['in_corn_window'] | flag
    print('已按默认全国窗口补齐 in_wheat_window / in_corn_window 标记。')

# 4) 空间耕地标记兜底：若缺失 is_cropland 列，则先置为 False 并提示
if 'is_cropland' not in fire_gdf.columns:
    fire_gdf['is_cropland'] = False
    print('提示：未检测到 is_cropland，已临时置为 False（可先运行耕地加载单元获得更准确判定）。')

# 5) 分类规则：空间（在耕地） AND 时间（在小麦或玉米收获窗口）
fire_gdf['ag_burning'] = fire_gdf['is_cropland'] & (fire_gdf['in_wheat_window'] | fire_gdf['in_corn_window'])

# 可选细化：标记具体作物窗口
fire_gdf['in_any_window'] = fire_gdf['in_wheat_window'] | fire_gdf['in_corn_window']
fire_gdf['ag_label'] = np.select(
    [fire_gdf['ag_burning'], fire_gdf['is_cropland'] & ~fire_gdf['in_any_window'], ~fire_gdf['is_cropland'] & fire_gdf['in_any_window']],
    ['likely_agri_burning', 'cropland_out_of_harvest', 'non_cropland_in_harvest'],
    default='other_or_uncertain'
)

# 6) 量化估计
summary = fire_gdf['ag_label'].value_counts().rename_axis('label').reset_index(name='count')
summary['percent'] = (summary['count'] / len(fire_gdf) * 100).round(2)
summary_out = RESULTS_TASK2 / 'ag_burning_summary.csv'
summary.to_csv(summary_out, index=False)
print('保存:', summary_out)
print(summary)

# 7) 导出点级分类（轻量列）
cols_export = ['datetime','year','month','season','is_cropland','in_wheat_window','in_corn_window','ag_label','geometry']
classified_points = fire_gdf[cols_export].copy()
classified_points_path = RESULTS_TASK2 / 'ag_burning_classified_points.geojson'
classified_points.to_file(classified_points_path, driver='GeoJSON')
print('保存:', classified_points_path)

# 8) 点图（分类别分层绘制，避免 color 与 column 参数冲突）
N = len(fire_gdf)
sample_rate = 1.0 if N <= 150000 else min(150000/N, 1.0)
plot_df = fire_gdf.sample(frac=sample_rate, random_state=42) if sample_rate < 1.0 else fire_gdf

plt.figure(figsize=(8,8))
base = None
try:
    if Path(COUNTY_SHP).exists():
        counties = gpd.read_file(COUNTY_SHP)
        if counties.crs is None:
            counties = counties.set_crs('EPSG:4326')
        elif counties.crs.to_epsg() != 4326:
            counties = counties.to_crs(4326)
        base = counties.boundary.plot(edgecolor='lightgray', linewidth=0.3)
except Exception:
    pass

palette = {
    'likely_agri_burning':'#D7191C',
    'cropland_out_of_harvest':'#FDAE61',
    'non_cropland_in_harvest':'#2C7FB8',
    'other_or_uncertain':'#AAAAAA'
}
for lab, color in palette.items():
    sub = plot_df[plot_df['ag_label'] == lab]
    if len(sub) > 0:
        sub.plot(ax=base, markersize=2, alpha=0.6, color=color, label=lab)
plt.title('火点分类（农业焚烧归因）')
plt.axis('off')
plt.legend(markerscale=4, fontsize=8, frameon=True)
plt.tight_layout()
points_map = RESULTS_TASK2 / 'ag_burning_points_map.png'
plt.savefig(points_map, dpi=150)
plt.show()
print('保存:', points_map)

# 9) 县域汇总与地图
try:
    if 'ag_label' not in fire_gdf.columns:
        raise ValueError('缺少 ag_label 列')
    if not Path(COUNTY_SHP).exists():
        raise FileNotFoundError(f'未找到县界文件: {COUNTY_SHP}')

    counties = gpd.read_file(COUNTY_SHP)
    if counties.crs is None:
        counties = counties.set_crs('EPSG:4326')
    elif counties.crs.to_epsg() != 4326:
        counties = counties.to_crs(4326)

    join = gpd.sjoin(fire_gdf[['geometry','ag_label']], counties, how='left', predicate='within')

    # 选择县级键
    key = None
    for c in ['county_id','adcode','code','id','County_ID','COUNTY_ID']:
        if c in join.columns:
            key = c
            break
    if key is None:
        counties = counties.reset_index().rename(columns={'index':'county_idx'})
        join = gpd.sjoin(fire_gdf[['geometry','ag_label']], counties, how='left', predicate='within')
        key = 'county_idx'

    # 每县“likely_agri_burning”计数
    cnt = join[join['ag_label']=='likely_agri_burning'].groupby(key).size().rename('ag_burn_count').reset_index()
    stats = counties.merge(cnt, on=key, how='left')
    stats['ag_burn_count'] = stats['ag_burn_count'].fillna(0).astype(int)

    out_csv = RESULTS_TASK2 / 'county_ag_burning_counts.csv'
    stats.drop(columns='geometry').to_csv(out_csv, index=False)
    print('保存:', out_csv)

    plt.figure(figsize=(8,8))
    ax = counties.boundary.plot(edgecolor='lightgray', linewidth=0.3)
    stats.plot(column='ag_burn_count', ax=ax, legend=True, cmap='OrRd', linewidth=0, alpha=0.9)
    plt.title('县域“农业焚烧”火点数')
    plt.axis('off')
    plt.tight_layout()
    county_map = RESULTS_TASK2 / 'county_ag_burning_map.png'
    plt.savefig(county_map, dpi=150)
    plt.show()
    print('保存:', county_map)
except Exception as e:
    print('县域汇总绘图失败：', e)


In [ ]:
# 收获季时间窗口（可选读取配置，否则使用默认全国窗口）
import pandas as pd
from datetime import datetime

from pathlib import Path  # 确保导入 Path

# 确保 CROP_DIR 已定义
if 'CROP_DIR' not in globals():
    if 'REPO_ROOT' in globals() and isinstance(REPO_ROOT, (Path, str)):
        if not isinstance(REPO_ROOT, Path):
            REPO_ROOT = Path(REPO_ROOT)
        CROP_DIR = REPO_ROOT / 'raw_data' / 'cropland_distribution_and_phenological_data'
    else:
        # 如果连 REPO_ROOT 都没有定义，尝试重新检测
        def detect_repo_root(start: Path = Path.cwd()) -> Path:
            for p in [start] + list(start.parents):
                if (p / 'raw_data').exists():
                    return p
            return start.parent if start.name == 'script' else start
        REPO_ROOT = detect_repo_root()
        CROP_DIR = REPO_ROOT / 'raw_data' / 'cropland_distribution_and_phenological_data'

CAL_DIR = CROP_DIR
cal_path = CAL_DIR / 'harvest_calendar.csv' if CAL_DIR.exists() else None
calendar_df = None
if cal_path and cal_path.exists():
    try:
        calendar_df = pd.read_csv(cal_path)
        print('使用自定义 harvest_calendar.csv')
        display(calendar_df.head())
    except Exception as e:
        print('读取 harvest_calendar.csv 失败，使用默认窗口。原因:', e)
else:
    print('未发现 harvest_calendar.csv，使用默认窗口。')

# 默认全国窗口（可按需微调）：
DEFAULT_WINDOWS = [
    # 小麦（冬小麦）：5/15–6/30
    {'crop':'wheat', 'start':(5,15), 'end':(6,30)},
    # 小麦（春小麦/北方局地）：8/01–9/15
    {'crop':'wheat', 'start':(8,1), 'end':(9,15)},
    # 玉米：9/01–11/15
    {'crop':'corn',  'start':(9,1), 'end':(11,15)}
]

def in_mmdd_window(dt: pd.Timestamp, start_m, start_d, end_m, end_d):
    if pd.isna(dt):
        return False
    m = dt.month; d = dt.day
    start = (start_m, start_d)
    end = (end_m, end_d)
    if start <= end:
        return (m, d) >= start and (m, d) <= end
    else:
        # 跨年窗口（这里不使用，但保留逻辑）
        return (m, d) >= start or (m, d) <= end

# 为每个火点计算 in_wheat_window / in_corn_window 标记
fire_gdf['in_wheat_window'] = False
fire_gdf['in_corn_window'] = False

if calendar_df is not None:
    # 若提供了日历，需将点映射到区域（县/省）后按区域窗口判定。
    # 这里先尝试县级映射；若列名中包含 region/adcode/code 等则按该键匹配。
    # 简化：按全国统一窗口作为兜底，区域规则留作后续扩展。
    print('当前版本按全国默认窗口 + 可选区域覆盖。若需严格区域化，请提供可匹配的区域编码列名，并说明对应县界字段。')

# 默认窗口打标
for win in DEFAULT_WINDOWS:
    crop = win['crop']
    sm, sd = win['start']
    em, ed = win['end']
    flag = fire_gdf['datetime'].apply(lambda t: in_mmdd_window(t, sm, sd, em, ed))
    if crop == 'wheat':
        fire_gdf['in_wheat_window'] = fire_gdf['in_wheat_window'] | flag
    elif crop == 'corn':
        fire_gdf['in_corn_window'] = fire_gdf['in_corn_window'] | flag

print('时间窗口标记完成。')

In [ ]:
# 任务2运行环境与数据保障（路径/变量自检）
# - 确保 REPO_ROOT/RAW_DIR/FIRE_DIR/COUNTY_SHP/RESULTS_DIR 存在
# - 定义 TASK2 结果目录 results/task2
# - 若 fire_gdf 未定义或为空，则在本单元内自动装载一次

from pathlib import Path
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# 1) 路径变量兜底
if 'REPO_ROOT' not in globals() or 'RAW_DIR' not in globals() or 'FIRE_DIR' not in globals() or 'COUNTY_SHP' not in globals() or 'RESULTS_DIR' not in globals():
    def detect_repo_root(start: Path = Path.cwd()) -> Path:
        for p in [start] + list(start.parents):
            if (p / 'raw_data').exists():
                return p
        return start.parent if start.name == 'script' else start
    REPO_ROOT = detect_repo_root()
    RAW_DIR = REPO_ROOT / 'raw_data'
    FIRE_DIR = RAW_DIR / 'satellite_fire_data'
    COUNTY_SHP = RAW_DIR / 'chn_county' / 'chn_county.shp'
    RESULTS_DIR = REPO_ROOT / 'results'
    RESULTS_DIR.mkdir(parents=True, exist_ok=True)
else:
    # 关键修复：确保所有路径变量都是 Path 对象
    if not isinstance(REPO_ROOT, Path):
        REPO_ROOT = Path(REPO_ROOT)
    if not isinstance(RAW_DIR, Path):
        RAW_DIR = Path(RAW_DIR)
    if not isinstance(FIRE_DIR, Path):
        FIRE_DIR = Path(FIRE_DIR)
    if not isinstance(COUNTY_SHP, Path):
        COUNTY_SHP = Path(COUNTY_SHP)
    if not isinstance(RESULTS_DIR, Path):
        RESULTS_DIR = Path(RESULTS_DIR)

# 2) 定义 Task2 独立结果目录
RESULTS_TASK2 = RESULTS_DIR / 'task2'
RESULTS_TASK2.mkdir(parents=True, exist_ok=True)
print('Task2 results dir:', RESULTS_TASK2)

# 3) 若缺少辅助函数，提供兜底实现
if 'find_column' not in globals():
    def find_column(cols, candidates):
        for cand in candidates:
            for col in cols:
                if col.lower() == cand.lower():
                    return col
        return None

if 'build_datetime' not in globals():
    def build_datetime(df):
        date_col = find_column(df.columns, ['acq_date', 'date', 'acq-date'])
        time_col = find_column(df.columns, ['acq_time', 'time', 'acq-time'])
        dt = None
        if date_col is not None and time_col is not None:
            def _fmt_time(t):
                try:
                    s = str(int(t)).zfill(4)
                    return f"{s[:2]}:{s[2:]}"
                except Exception:
                    return None
            times = df[time_col].apply(_fmt_time)
            dt = pd.to_datetime(df[date_col].astype(str) + ' ' + times.astype(str), errors='coerce')
        elif date_col is not None:
            dt = pd.to_datetime(df[date_col], errors='coerce')
        else:
            ts_col = find_column(df.columns, ['timestamp'])
            if ts_col is not None:
                dt = pd.to_datetime(df[ts_col], errors='coerce')
        if dt is None:
            raise ValueError('未能识别日期/时间列，请检查原始 CSV 列名。')
        df['datetime'] = dt
        df['date'] = df['datetime'].dt.date
        df['year'] = df['datetime'].dt.year
        df['month'] = df['datetime'].dt.month
        df['day'] = df['datetime'].dt.day
        # 简化季节映射
        MONTH2SEASON = {12:'Winter',1:'Winter',2:'Winter',3:'Spring',4:'Spring',5:'Spring',6:'Summer',7:'Summer',8:'Summer',9:'Autumn',10:'Autumn',11:'Autumn'}
        df['season'] = df['month'].map(MONTH2SEASON)
        return df

if 'to_geodf' not in globals():
    def to_geodf(df):
        lon_col = find_column(df.columns, ['longitude', 'lon', 'long'])
        lat_col = find_column(df.columns, ['latitude', 'lat'])
        if lon_col is None or lat_col is None:
            raise ValueError('未找到经纬度列（longitude/latitude 或 lon/lat）。')
        geometry = [Point(xy) for xy in zip(df[lon_col].astype(float), df[lat_col].astype(float))]
        gdf = gpd.GeoDataFrame(df.copy(), geometry=geometry, crs='EPSG:4326')
        return gdf

# 4) 若 fire_gdf 未就绪，则自动装载
if 'fire_gdf' not in globals() or fire_gdf is None or len(fire_gdf) == 0:
    from glob import glob
    if not isinstance(FIRE_DIR, Path):
        FIRE_DIR = Path(FIRE_DIR)
    
    csv_paths = sorted(FIRE_DIR.glob('modis_*_china.csv'))
    csv_paths = [str(p) for p in csv_paths]  # 转为字符串列表以兼容 pd.read_csv

    frames = []
    for p in csv_paths:
        try:
            df = pd.read_csv(p)
            df = build_datetime(df)
            lon_col = find_column(df.columns, ['longitude', 'lon', 'long'])
            lat_col = find_column(df.columns, ['latitude', 'lat'])
            if lon_col and lat_col:
                df = df[(df[lon_col].between(70, 140)) & (df[lat_col].between(15, 55))]
            try:
                year_in_name = int(Path(p).stem.split('_')[1])
                df['year_from_fname'] = year_in_name
            except Exception:
                df['year_from_fname'] = df['year']
            frames.append(df)
        except Exception as e:
            print(f'读取失败 {p}: {e}')
    fire_df = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
    if not fire_df.empty:
        fire_gdf = to_geodf(fire_df).set_crs('EPSG:4326', allow_override=True)
    else:
        fire_gdf = gpd.GeoDataFrame(columns=['datetime','year','month','season','geometry'], geometry='geometry', crs='EPSG:4326')
print('fire_gdf 就绪，记录数:', len(fire_gdf))

## 任务2与改进版之间的说明（为什么原始结果被低估、改进的动机）

本研究的核心问题是：在多大程度上可将观测到的火点归因为收获后秸秆焚烧。原始任务2按“全国火点 × 耕地覆盖 × 收获窗口”的框架计算，但本仓库目前提供的耕地分布与物候数据仅覆盖黑龙江省（2010–2019），未覆盖全国其他省份。

这导致两个直接后果：
- 覆盖错配：当用“仅黑龙江的作物栅格”去匹配“全国范围的火点”时，黑龙江以外的火点大多被判为“非耕地”。
- 归因被系统性低估：许多本应属于“耕地附近”的火点被划入“非耕地/不确定”，使得“likely_agri_burning”的全国占比被压低到 ~1.77%。

此外，还有两点会进一步压低归因：
- 未考虑“田边焚烧”：若不对耕地做邻近缓冲，田块边界附近的焚烧点经常被误判为非耕地。
- 收获窗口未分省：全国统一窗口无法反映南北与地貌差异，时间匹配会在不同地区出现偏宽/偏窄。

因此，我们增加了一个“改进版（轻量）”以提供“在数据覆盖区域内”的可信结论，并为未来全国化铺路：
1) 覆盖感知：先用县域边界筛选出“黑龙江省内”的火点，只在数据覆盖区域内计算归因比例；
2) 省级窗口（近似，可替换为更精细的省历）：为黑龙江设置小麦/玉米的收获时间；
3) 耕地邻近缓冲（~500m 等效）：对作物栅格做像素邻域搜索，捕捉田边焚烧；
4) 重新分类并导出“覆盖区域内”的地图与统计。

改进后的黑龙江区域结果：
- likely_agri_burning ≈ 27.57%（与文献 15%–30% 区间一致）
- 相比原始全国口径的 1.77%，这更能反映方法在“数据覆盖到位”的地区的真实性能。

接下来若要得到“全国口径”的可信结论，建议：
- 补充全国耕地/作物分布数据（或至少华北、华东等重点区），直接套用本改进版框架；
- 或分省逐步补齐覆盖，最后合并各省的“覆盖区域内估计”形成全国估计。


In [ ]:
# 改进实现：以黑龙江为例的覆盖感知 + 省级窗口 + 栅格邻域缓冲
import os, math, warnings, json
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import box
import rasterio
from rasterio.windows import Window
from rasterio import transform as rio_transform
import matplotlib.pyplot as plt
from matplotlib import rcParams as _rc
warnings.filterwarnings('ignore')

# 0) 兜底：获取 fire_gdf 与 counties
if 'fire_gdf' not in globals():
    # 如果内存中没有，就尝试从之前导出的点集加载（若存在）
    pts_csv = Path(RESULTS_TASK2)/'ag_burning_classified_points.geojson'
    if pts_csv.exists():
        fire_gdf = gpd.read_file(pts_csv)
    else:
        raise RuntimeError('找不到 fire_gdf，请先运行上游生成火点数据的单元。')
if 'counties' not in globals():
    shp = Path(RAW_DIR)/'chn_county'
    shp = next((shp/f for f in os.listdir(shp) if f.endswith('.shp')), None)
    if shp is None:
        raise RuntimeError('找不到县域矢量数据。')
    counties = gpd.read_file(shp)
if fire_gdf.crs is None:
    fire_gdf = fire_gdf.set_crs(4326)
elif fire_gdf.crs.to_epsg() != 4326:
    fire_gdf = fire_gdf.to_crs(4326)
if counties.crs is None:
    counties = counties.set_crs(4326)
elif counties.crs.to_epsg() != 4326:
    counties = counties.to_crs(4326)

# 1) 锁定黑龙江省区域多边形
prov_col = '省级' if '省级' in counties.columns else ('NAME_1' if 'NAME_1' in counties.columns else None)
if prov_col is None:
    raise RuntimeError('无法识别县域数据中的省级字段（期望 省级 或 NAME_1）。')

def _is_hlj(x):
    s = str(x)
    return ('黑龙江' in s) or ('Heilongjiang' in s) or ('Hēi Lóng Jiāng' in s)

hlj_poly = counties[counties[prov_col].apply(_is_hlj)].dissolve().geometry.iloc[0]

# 2) 仅取黑龙江内的火点（覆盖感知）
# 使用几何关系直接筛选，避免构造临时 GeoDataFrame 导致的类型检查/兼容性问题
hlj_poly = counties[counties[prov_col].apply(_is_hlj)].dissolve().geometry.unary_union
# unary_union 会返回一个单一的 Geometry 对象（Polygon/MultiPolygon），适合用于 .within()
hlj_fire = fire_gdf[fire_gdf.geometry.within(hlj_poly)].copy()
hlj_fire = hlj_fire.drop(columns=[c for c in hlj_fire.columns if c.startswith('index_')], errors='ignore')
print(f'黑龙江省内火点数: {len(hlj_fire):,}')

# 3) 定义黑龙江作物收获窗口（近似）
def _parse_date_col(df):
    for c in ['ACQ_DATE','date','DATE','acq_date']:
        if c in df.columns:
            return pd.to_datetime(df[c], errors='coerce')
    # 如果已有 year/month/day 列也可拼装
    if set(['year','month','day']).issubset(df.columns):
        return pd.to_datetime(df[['year','month','day']].rename(columns={'year':'Y','month':'M','day':'D'}).assign(Y=lambda d:d['Y'].astype(int),M=lambda d:d['M'].astype(int),D=lambda d:d['D'].astype(int)).apply(lambda r: f"{int(r['Y']):04d}-{int(r['M']):02d}-{int(r['D']):02d}", axis=1), errors='coerce')
    raise RuntimeError('无法从火点数据中解析日期列。')

hlj_fire = hlj_fire.copy()
# 强制转换日期为 datetime
if 'date' in hlj_fire.columns:
    hlj_fire['date'] = pd.to_datetime(hlj_fire['date'], errors='coerce')
else:
    hlj_fire['date'] = _parse_date_col(hlj_fire)

hlj_fire['month'] = hlj_fire['date'].dt.month
hlj_fire['day'] = hlj_fire['date'].dt.day
hlj_fire['year'] = hlj_fire['date'].dt.year

# 省级窗口（可根据资料进一步细化）
# 春小麦（HLJ）收割大致在 8/1-9/10，玉米在 9/25-11/10

def in_window(m, d, sm, sd, em, ed):
    import datetime as _dt
    try:
        t = _dt.date(2020, int(m), int(d))
        a = _dt.date(2020, sm, sd)
        b = _dt.date(2020, em, ed)
        return (a <= t <= b)
    except Exception:
        return False

hlj_fire['in_wheat_window_hlj'] = [in_window(m, d, 8, 1, 9, 10) for m, d in zip(hlj_fire['month'], hlj_fire['day'])]
hlj_fire['in_corn_window_hlj']  = [in_window(m, d, 9, 25, 11, 10) for m, d in zip(hlj_fire['month'], hlj_fire['day'])]
hlj_fire['in_harvest_hlj'] = hlj_fire['in_wheat_window_hlj'] | hlj_fire['in_corn_window_hlj']

# 4) 栅格邻域缓冲：针对 HLJ 每年小麦/玉米栅格，在像素邻域内搜索
CROPLAND_BUFFER_M = int(os.environ.get('CROPLAND_BUFFER_M', 500))
CROP_DIR = Path(RAW_DIR)/'cropland_distribution_and_phenological_data'

def open_raster_safe(path: Path):
    if not path.exists():
        return None, None, None, None
    src = rasterio.open(path)
    arr = src.read(1)
    ndv = src.nodata if src.nodata is not None else np.nan
    finite = arr[np.isfinite(arr)] if np.isnan(ndv) else arr[arr != ndv]
    vmax = float(np.nanmax(finite)) if finite.size>0 else 0.0
    # 自适应阈值
    thr = 0.2 if vmax<=1.0 else (20.0 if vmax>=100.0 else 0.2*vmax)
    return src, arr, ndv, thr
def meters_per_pixel(src):
    # 估算像素大小：经纬度转米（按影像中心纬度）
    T = src.transform
    px_deg_x = abs(T.a)
    px_deg_y = abs(T.e)
    # 取影像中心纬度
    c = rio_transform.xy(T, src.height//2, src.width//2)
    lat = c[1] if isinstance(c, (tuple, list)) else 45.0
    m_per_deg_lat = 111320.0
    m_per_deg_lon = 111320.0 * math.cos(math.radians(lat))
    mpx_x = px_deg_x * m_per_deg_lon
    mpx_y = px_deg_y * m_per_deg_lat
    return (mpx_x + mpx_y)/2.0
    return (mpx_x + mpx_y)/2.0

def buffered_hit_for_points(df_year, rasters):
    # rasters: list of (src, arr, ndv, thr) for maize & wheat
    if len(df_year)==0:
        return np.array([], dtype=bool)
    hits = np.zeros(len(df_year), dtype=bool)
    for src, arr, ndv, thr in rasters:
        if src is None:
            continue
        mpp = meters_per_pixel(src)
        rad_px = max(1, int(round(CROPLAND_BUFFER_M / max(mpp,1e-6))))
        # 将经纬度转为行列
        rows_cols = [src.index(lon, lat) for lon, lat in zip(df_year.geometry.x, df_year.geometry.y)]
        h, w = arr.shape
        for i, (r, c) in enumerate(rows_cols):
            if r<0 or r>=h or c<0 or c>=w:
                continue
            r0, r1 = max(0, r-rad_px), min(h, r+rad_px+1)
            c0, c1 = max(0, c-rad_px), min(w, c+rad_px+1)
            sub = arr[r0:r1, c0:c1]
            if np.isnan(ndv):
                valid = sub[np.isfinite(sub)]
            else:
                valid = sub[sub!=ndv]
            if valid.size==0:
                continue
            if np.nanmax(valid) > thr:
                hits[i] = True
    return hits

# 逐年处理 HLJ 火点
hlj_fire = hlj_fire.copy()
hlj_fire['is_cropland'] = hlj_fire.get('is_cropland', False)
buf_hits_total = np.zeros(len(hlj_fire), dtype=bool)
years = sorted([int(y) for y in pd.unique(hlj_fire['year'].dropna())])
for y in years:
    sub_idx = np.where(hlj_fire['year'].values==y)[0]
    sub = hlj_fire.iloc[sub_idx]
    maize = CROP_DIR/f'heilongjiang_maize_ma_{y}.tif'
    wheat = CROP_DIR/f'heilongjiang_wheat_ma_{y}.tif'
    rasters = []
    for pth in [maize, wheat]:
        src, arr, ndv, thr = open_raster_safe(pth)
        rasters.append((src, arr, ndv, thr))
    hits = buffered_hit_for_points(sub, rasters)
    buf_hits_total[sub_idx] = hits
for src, _, _, _ in rasters:
    if src is not None:
        try:
            src.close()
        except Exception as e:
            print(f"关闭 raster 时出错: {e}")

hlj_fire['is_cropland_buffered'] = hlj_fire['is_cropland'] | buf_hits_total

# 5) 改进分类（HLJ 内）

def classify_row(row):
    on_crop = bool(row.get('is_cropland_buffered', False))
    in_harv = bool(row.get('in_harvest_hlj', False))
    if on_crop and in_harv:
        return 'likely_agri_burning'
    if on_crop and not in_harv:
        return 'cropland_out_of_harvest'
    if (not on_crop) and in_harv:
        return 'non_cropland_in_harvest'
    return 'other_or_uncertain'

hlj_fire['label_improved'] = hlj_fire.apply(classify_row, axis=1)
summary = (hlj_fire['label_improved'].value_counts().rename_axis('label').reset_index(name='count'))
summary['percent'] = (summary['count']/summary['count'].sum()*100).round(2)
print('HLJ 改进版汇总:')
print(summary)

# 6) 导出与地图
out_dir = RESULTS_TASK2
out_dir.mkdir(parents=True, exist_ok=True)
summary_path = out_dir/'ag_burning_summary_hlj_improved.csv'
summary.to_csv(summary_path, index=False)
print('保存:', summary_path)

# 点图
try:
    _rc['font.sans-serif'] = ['Microsoft YaHei','SimHei','Noto Sans CJK SC','Arial'] + _rc.get('font.sans-serif', [])
    base = counties.plot(color='lightgrey', edgecolor='white', linewidth=0.2, figsize=(7,7))
    palette = {'likely_agri_burning':'#ff6b6b','cropland_out_of_harvest':'#ffa07a','non_cropland_in_harvest':'#8ecae6','other_or_uncertain':'#c0c0c0'}
    for lab in ['other_or_uncertain','non_cropland_in_harvest','cropland_out_of_harvest','likely_agri_burning']:
        sub = hlj_fire[hlj_fire['label_improved']==lab]
        if len(sub)==0:
            continue
        sub.plot(ax=base, markersize=2 if lab!='likely_agri_burning' else 6, alpha=0.6 if lab!='likely_agri_burning' else 0.9, color=palette.get(lab,'grey'))
    plt.title('黑龙江 火点分类（改进版）')
    points_map = out_dir/'ag_burning_points_map_hlj_improved.png'
    plt.savefig(points_map, dpi=180, bbox_inches='tight')
    plt.show()
    plt.close()
    print('保存:', points_map)
except Exception as e:
    print('绘制点图失败:', e)

# 县域聚合与底图
try:
    key = '县级码' if '县级码' in counties.columns else ('code' if 'code' in counties.columns else None)
    name_key = '县级' if '县级' in counties.columns else ('NAME_3' if 'NAME_3' in counties.columns else None)
    if key is None or name_key is None:
        raise RuntimeError('无法识别县级编码/名称字段。')
    # 仅保留 HLJ 县域
    hlj_counties = counties[counties[prov_col].apply(_is_hlj)].copy()
    # 需要火点关联到县级：若先前没有，则做一次 within 关联
    if 'county_code' not in hlj_fire.columns:
        join = gpd.sjoin(hlj_fire, hlj_counties[[key, name_key, 'geometry']].rename(columns={key:'county_code', name_key:'county_name'}), predicate='within', how='left')
    else:
        join = hlj_fire
    cnt = (join[join['label_improved']=='likely_agri_burning']
           .groupby('county_code').size().rename('ag_burn_count').reset_index())
    out_csv = out_dir/'county_ag_burning_counts_hlj_improved.csv'
    cnt.to_csv(out_csv, index=False)
    print('保存:', out_csv)
    # 画县域热力图
    hlj_counties = hlj_counties.merge(cnt, left_on=key, right_on='county_code', how='left')
    hlj_counties['ag_burn_count'] = hlj_counties['ag_burn_count'].fillna(0)
    ax = hlj_counties.plot(column='ag_burn_count', cmap='OrRd', linewidth=0.1, edgecolor='white', legend=True, figsize=(8,6))
    plt.title('黑龙江 县域“农业焚烧”火点数（改进版）')
    county_map = out_dir/'county_ag_burning_map_hlj_improved.png'
    plt.savefig(county_map, dpi=180, bbox_inches='tight')
    plt.show()
    plt.close()
    print('保存:', county_map)
except Exception as e:
    print('县域热力图失败:', e)

## 任务3：长期时空趋势（黑龙江 2010–2019）
本节仅聚焦黑龙江省（因数据覆盖局限），基于“改进版”的农业焚烧判定结果，评估：
- 县级 2010–2019 年度“likely_agri_burning”趋势（Mann–Kendall 与 Sen 斜率）
- 县级是否存在显著拐点（Pettitt 非参变点检验，单一突变）
- 省级年度总量序列与潜在拐点
所有结果保存至 `results/task3/`，并在 Notebook 内联显示关键图。

In [ ]:
# 任务3实现：县级趋势与拐点（HLJ 2010–2019）
import os, math, warnings
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import rcParams as _rc
warnings.filterwarnings('ignore')

# 输出目录（定位到仓库根下的 results/task3）
if 'REPO_ROOT' in globals():
    _ROOT = REPO_ROOT
else:
    _ROOT = Path.cwd().resolve()
    
# ⚠️ 确保 _ROOT 是 Path 对象
if not isinstance(_ROOT, Path):
    _ROOT = Path(_ROOT)

if _ROOT.name.lower() == 'script':
    _ROOT = _ROOT.parent
RESULTS_TASK3 = (_ROOT / 'results' / 'task3')
RESULTS_TASK3.mkdir(parents=True, exist_ok=True)
print('Task3 输出目录:', RESULTS_TASK3)

# 复用已有 counties、hlj_fire（若不存在则快速构建 HLJ 子集）
def _ensure_hlj_fire_with_labels():
    # ⚠️ 修复：移除 global 声明，避免未定义变量
    # global hlj_fire, counties
    
    # 1) counties
    if 'counties' not in globals():
        # ⚠️ 修复：确保 shp_dir 是 Path 对象并检查存在性
        shp_dir = _ROOT / 'raw_data' / 'chn_county'
        if not isinstance(shp_dir, Path):
            shp_dir = Path(shp_dir)
        
        if not shp_dir.exists():
            raise FileNotFoundError(f"县界目录不存在: {shp_dir}")
        
        # ⚠️ 修复：安全地查找 shapefile
        shp_files = list(shp_dir.glob('*.shp'))
        if not shp_files:
            raise FileNotFoundError(f"在 {shp_dir} 中未找到 .shp 文件")
        
        shp = shp_files[0]  # 取第一个找到的 shp 文件
        counties_local = gpd.read_file(shp)
    else:
        # ⚠️ 修复：确保 counties 是有效的 GeoDataFrame
        counties_local = globals()['counties']  # 使用 globals() 安全访问
    if counties_local.crs is None:
        counties_local = counties_local.set_crs(4326)
    elif counties_local.crs.to_epsg()!=4326:
        counties_local = counties_local.to_crs(4326)
    prov_col = '省级' if '省级' in counties_local.columns else ('NAME_1' if 'NAME_1' in counties_local.columns else None)
    if prov_col is None:
        # ⚠️ 修复：如果没有找到省级列，尝试其他常见列名
        possible_cols = ['province', 'PROVINCE', 'Province', 'name_1', 'NAME_1']
        for col in possible_cols:
            if col in counties_local.columns:
                prov_col = col
                break
        if prov_col is None:
            raise ValueError("无法找到省级列，请检查县界文件的列名")
        
    def _is_hlj(x):
        s=str(x); return ('黑龙江' in s) or ('Heilongjiang' in s) or ('Hēi Lóng Jiāng' in s)
    
    # ⚠️ 修复：添加安全检查，确保有黑龙江数据
    hlj_mask = counties_local[prov_col].apply(_is_hlj)
    if not hlj_mask.any():
        raise ValueError("在县界数据中未找到黑龙江相关区域")
    
    hlj_poly = counties_local[hlj_mask].dissolve().geometry.iloc[0]
    
    # 2) fire
    # ⚠️ 修复：安全检查 hlj_fire 变量
    if 'hlj_fire' in globals() and globals()['hlj_fire'] is not None and hasattr(globals()['hlj_fire'], 'columns') and ('label_improved' in globals()['hlj_fire'].columns):
        return globals()['hlj_fire'], counties_local, prov_col
    
    if 'fire_gdf' in globals():
        fire_local = globals()['fire_gdf'].copy()
    else:
        # 从改进版或通用导出回读（可能不含 label_improved，仅用于定位 HLJ 范围）
        gpath = _ROOT / 'results' / 'task2' / 'ag_burning_classified_points.geojson'
        if not gpath.exists():
            raise RuntimeError('找不到 fire_gdf，请先运行任务2改进版或生成火点数据单元。')
        fire_local = gpd.read_file(gpath)
    if fire_local.crs is None:
        fire_local = fire_local.set_crs(4326)
    elif fire_local.crs.to_epsg()!=4326:
        fire_local = fire_local.to_crs(4326)
    
    # ⚠️ 修复：使用正确的 GeoDataFrame 构造方式
    hlj_region = gpd.GeoDataFrame([{'geometry': hlj_poly}], crs=4326)
    hlj_fire_local = gpd.sjoin(fire_local, hlj_region, predicate='within', how='inner')
    
    hlj_fire_local = hlj_fire_local.drop(columns=[c for c in hlj_fire_local.columns if c.startswith('index_')], errors='ignore')
    
    # 兜底：确保 year 存在
    if 'date' in hlj_fire_local.columns:
        hlj_fire_local['date'] = pd.to_datetime(hlj_fire_local['date'], errors='coerce')
    elif 'ACQ_DATE' in hlj_fire_local.columns:
        hlj_fire_local['date'] = pd.to_datetime(hlj_fire_local['ACQ_DATE'], errors='coerce')
    else:
        raise RuntimeError('无法解析日期列以计算年度。')
    hlj_fire_local['year'] = hlj_fire_local['date'].dt.year
    
    # 若不存在改进标签，则尽量复用已有列（降级策略：in_harvest 或原 label）
    if 'label_improved' not in hlj_fire_local.columns:
        if 'label' in hlj_fire_local.columns:
            hlj_fire_local['label_improved'] = hlj_fire_local['label']
        else:
            in_cols = [c for c in hlj_fire_local.columns if 'in_harvest' in c]
            if len(in_cols)>0:
                col = in_cols[0]
                hlj_fire_local['label_improved'] = np.where(hlj_fire_local[col].astype(bool),'likely_agri_burning','other_or_uncertain')
            else:
                raise RuntimeError('找不到可用的农业焚烧标签（label_improved/label/in_harvest*）。')
    
    return hlj_fire_local, counties_local, prov_col

hlj_fire3, counties3, prov_col3 = _ensure_hlj_fire_with_labels()
print('HLJ 点数量:', len(hlj_fire3))

# 关联县级编码
key = '县级码' if '县级码' in counties3.columns else ('code' if 'code' in counties3.columns else None)
name_key = '县级' if '县级' in counties3.columns else ('NAME_3' if 'NAME_3' in counties3.columns else None)

# ⚠️ 修复：添加安全检查
if key is None:
    possible_keys = ['COUNTY_CODE', 'adcode', 'ADCODE', 'County_Code', 'county_code', 'code']
    for col in possible_keys:
        if col in counties3.columns:
            key = col
            break
    if key is None:
        raise ValueError("无法找到县级编码列，请检查县界文件的列名")

if name_key is None:
    possible_names = ['COUNTY_NAME', 'name', 'NAME', 'County_Name', 'county_name', 'NAME_3']
    for col in possible_names:
        if col in counties3.columns:
            name_key = col
            break
    if name_key is None:
        raise ValueError("无法找到县级名称列，请检查县界文件的列名")

hlj_counties = counties3[counties3[prov_col3].apply(lambda x: ('黑龙江' in str(x)) or ('Heilongjiang' in str(x)) or ('Hēi Lóng Jiāng' in str(x)))].copy()
join = gpd.sjoin(hlj_fire3, hlj_counties[[key, name_key, 'geometry']].rename(columns={key:'county_code', name_key:'county_name'}), predicate='within', how='left')

# 县-年 计数（仅 likely_agri_burning）
sub = join[join['label_improved']=='likely_agri_burning'].copy()
cnt = (sub.groupby(['county_code','county_name','year']).size().rename('count').reset_index())
# 填补 2010–2019 缺失年份为 0
all_years = list(range(2010, 2020))
rows = []
for code, g in cnt.groupby('county_code'):
    name = g['county_name'].iloc[0] if 'county_name' in g.columns else None
    y2c = dict(zip(g['year'], g['count']))
    for y in all_years:
        rows.append({'county_code':code,'county_name':name,'year':y,'count':int(y2c.get(y,0))})
cnt_full = pd.DataFrame(rows)
out_counts = RESULTS_TASK3/'hlj_county_year_likely_agri_counts.csv'
cnt_full.to_csv(out_counts, index=False, encoding='utf-8-sig')
print('保存:', out_counts)

# 统计学工具：Mann-Kendall、Sen's slope、Pettitt
from math import sqrt
from itertools import combinations

def mk_test(x):
    x = np.asarray(x, dtype=float)
    n = len(x)
    S = 0
    for i in range(n-1):
        S += np.sum(np.sign(x[i+1:]-x[i]))
    varS = (n*(n-1)*(2*n+5))/18
    z = 0
    if S>0: z = (S-1)/sqrt(varS)
    elif S<0: z = (S+1)/sqrt(varS)
    from math import erf
    p = 2*(1-0.5*(1+erf(abs(z)/sqrt(2))))
    trend = 'increasing' if z>0 and p<0.05 else ('decreasing' if z<0 and p<0.05 else 'no-trend')
    return {'S':S,'z':z,'p':p,'trend':trend}

def sen_slope(x, years):
    pairs = []
    for (i, xi),(j, xj) in combinations(list(enumerate(x)), 2):
        if years[j]==years[i]:
            continue
        pairs.append((xj - xi)/ (years[j]-years[i]))
    if len(pairs)==0:
        return 0.0
    return float(np.median(pairs))

def pettitt_test(x):
    x = np.asarray(x, dtype=float)
    n = x.size
    if n<2:
        return (None, 1.0)
    U = np.zeros(n)
    for t in range(n):
        s = 0
        for i in range(n):
            for j in range(i+1):
                if j<=t and i>t:
                    s += np.sign(x[i]-x[j])
        U[t] = s
    K = int(np.argmax(np.abs(U)))
    Kmax = np.max(np.abs(U))
    p = 2*np.exp((-6*(Kmax**2))/(n**3+n**2))
    return (K if p<0.05 else None, float(p))

# 对每个县做趋势与拐点
rows = []
for code, g in cnt_full.groupby('county_code'):
    g = g.sort_values('year')
    y = g['year'].tolist()
    v = g['count'].astype(float).tolist()
    mk = mk_test(v)
    slope = sen_slope(v, y)
    cp_idx, cp_p = pettitt_test(v)
    cp_year = int(y[cp_idx]) if cp_idx is not None else None
    pre_mean = float(np.mean(v[:cp_idx+1])) if cp_idx is not None else np.nan
    post_mean = float(np.mean(v[cp_idx+1:])) if cp_idx is not None else np.nan
    rows.append({'county_code':code,'county_name':g['county_name'].iloc[0],
                 'mk_trend':mk['trend'],'mk_p':mk['p'],'sen_slope':slope,
                 'change_year':cp_year,'change_p':cp_p,'pre_mean':pre_mean,'post_mean':post_mean})
summary = pd.DataFrame(rows)
out_summary = RESULTS_TASK3/'hlj_county_trend_summary.csv'
summary.to_csv(out_summary, index=False, encoding='utf-8-sig')
print('保存:', out_summary)

# 省级年度序列与拐点
prov_series = cnt_full.groupby('year')['count'].sum().reset_index()
prov_series_path = RESULTS_TASK3/'hlj_province_series.csv'
prov_series.to_csv(prov_series_path, index=False, encoding='utf-8-sig')
cp_idx_p, cp_pv = pettitt_test(prov_series['count'].values)
cp_year_p = int(prov_series['year'].iloc[cp_idx_p]) if cp_idx_p is not None else None
print('省级总量：变点年份=', cp_year_p, ' p=', round(cp_pv,4))

# 可视化
_rc['font.sans-serif'] = ['Microsoft YaHei','SimHei','Noto Sans CJK SC','Arial'] + _rc.get('font.sans-serif', [])
fig, ax = plt.subplots(1,1, figsize=(7,4))
ax.plot(prov_series['year'], prov_series['count'], marker='o')
if cp_year_p is not None:
    ax.axvline(cp_year_p, color='red', ls='--', label=f'Change {cp_year_p}')
ax.set_title('黑龙江 全省 likely_agri_burning 年度总量（2010–2019）')
ax.set_xlabel('Year'); ax.set_ylabel('Count')
ax.legend(loc='upper left')
png = RESULTS_TASK3/'hlj_province_series.png'
plt.savefig(png, dpi=180, bbox_inches='tight'); plt.show(); plt.close()
print('保存:', png)

# 地图：Sen 斜率与变点年份
gmap = hlj_counties.merge(summary, left_on=key, right_on='county_code', how='left')
fig, ax = plt.subplots(1,1, figsize=(8,6))
gmap.plot(column='sen_slope', cmap='RdBu_r', legend=True, linewidth=0.1, edgecolor='white', ax=ax, vmin=-40, vmax=40)
ax.set_title('黑龙江 县域 Sen 斜率（likely_agri_burning，2010–2019）')
png = RESULTS_TASK3/'hlj_sen_slope_map.png'
plt.savefig(png, dpi=180, bbox_inches='tight'); plt.show(); plt.close()
print('保存:', png)

# 变点年份：若没有显著变点，给出信息化底图
fig, ax = plt.subplots(1,1, figsize=(8,6))
if gmap['change_year'].notna().sum() > 0:
    gmap.plot(column='change_year', cmap='viridis', legend=True, linewidth=0.1, edgecolor='white', ax=ax)
    ax.set_title('黑龙江 县域 变点年份（Pettitt，p<0.05 为显著）')
else:
    gmap.plot(color='#f2f2f2', edgecolor='white', linewidth=0.1, ax=ax)
    ax.set_title('黑龙江 县域：未检测到显著变点（p<0.05）')
    # 输出说明文件
    with open(RESULTS_TASK3/'no_significant_change_points.txt', 'w', encoding='utf-8') as f:
        f.write('Pettitt 检验在县域尺度未检测到显著变点（p<0.05）。\n')
png = RESULTS_TASK3/'hlj_change_year_map.png'
plt.savefig(png, dpi=180, bbox_inches='tight'); plt.show(); plt.close()
print('保存:', png)

In [ ]:
# 任务3小结与概览（统计汇总与TOP榜）
import pandas as pd, numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import rcParams as _rc
_rc['font.sans-serif'] = ['Microsoft YaHei','SimHei','Noto Sans CJK SC','Arial'] + _rc.get('font.sans-serif', [])

ROOT = Path.cwd()
if ROOT.name.lower()=='script':
    ROOT = ROOT.parent
TASK3 = ROOT/'results'/'task3'
df = pd.read_csv(TASK3/'hlj_county_trend_summary.csv')
# 概览计数
overview = (df['mk_trend'].value_counts().rename_axis('mk_trend').reset_index(name='count'))
overview['percent'] = (overview['count']/overview['count'].sum()*100).round(1)
print('MK 趋势概览:\n', overview)
# Top榜（按 Sen 斜率）
top_dec = df.sort_values('sen_slope').head(10)[['county_name','sen_slope','mk_trend','mk_p']]
top_inc = df.sort_values('sen_slope', ascending=False).head(10)[['county_name','sen_slope','mk_trend','mk_p']]
top_dec.to_csv(TASK3/'top10_decreasing_counties.csv', index=False, encoding='utf-8-sig')
top_inc.to_csv(TASK3/'top10_increasing_counties.csv', index=False, encoding='utf-8-sig')
print('保存: top10_decreasing_counties.csv, top10_increasing_counties.csv')

# 斜率分布图
fig, ax = plt.subplots(1,1, figsize=(7,4))
ax.hist(df['sen_slope'].fillna(0), bins=25, color='#6baed6', edgecolor='white')
ax.set_title('县域 Sen 斜率分布（likely_agri_burning，2010–2019）')
ax.set_xlabel('Sen slope (count per year)'); ax.set_ylabel('Number of counties')
png = TASK3/'hlj_sen_slope_hist.png'
plt.savefig(png, dpi=180, bbox_inches='tight'); plt.show(); plt.close()
print('保存:', png)
overview.to_csv(TASK3/'hlj_task3_overview.csv', index=False, encoding='utf-8-sig')

## 任务3结果解读与改进建议（黑龙江 2010–2019）
本节对上方产物做一个可交付级的简要结论，并说明改进方向。

结论要点：
- 省级年度总量：2014–2015 达到峰值，2016 起显著回落；Pettitt 检验未给出显著的单一突变年份（p≈0.495），但走势与禁烧治理强化的时间线相符。
- 县级趋势（likely_agri_burning，2010–2019）：
  - Mann–Kendall：多数县“无显著趋势”（~96%），有少量县显著下降，极少数显著上升。
  - Sen 斜率空间格局：以负斜率为主，显示“多数县在下降”，与省级走势一致。
- 变点检测（县级）：严格阈值 p<0.05 下，多数县未检出显著变点；说明变化更像“缓慢下降”或“多段渐变”，而非统一时间点的突变。

为何显著性偏弱：
- 时间样本较短（10年）且波动较大（2010–2015 高位、2016 以后快速回落），在非参检验中较难判定为“单调显著”。
- 2018–2019 点数极低，拉低末期，导致分布非正态且存在零膨胀，对 MK 检验的功效不利。

与文献的一致性：
- 文献普遍报告“禁烧政策后焚烧显著减少”。本结果的省级曲线与大多数研究一致；县域 Sen 斜率以负为主也支撑“总体下降”。
- 但“显著县比例较低”与样本年限、零膨胀和年内时序聚集有关，需要采用更针对性的统计方法来增强检出力。

改进建议（可选增强，按优先级）：
1) 季节化/窗口聚焦的趋势：仅对“收获窗口内”的农业焚烧计数做年度序列（已具备标签，可直接派生），减少非农业/非窗口噪声。
2) 零膨胀稳健检验：采用季节 Kendall、预白化 MK（Hamed–Rao）、或基于泊松/负二项的广义线性趋势检验；给出 Sen 斜率的 bootstrap 置信区间。
3) 频率与强度拆分：分别对“火点次数”和“高强度阈值（如FRP分位）”做趋势与变点，识别强度结构性变化。
4) 变点更丰富的模型：尝试 binary segmentation / PELT / Bai–Perron 多变点检测（年数虽少，但可作启发式），并与政策节点对照。
5) 归一化口径：用县域耕地面积归一化（若可获得），输出“单位耕地的焚烧强度”趋势，减小行政区面积差影响。

交付物索引（results/task3）：
- hlj_county_year_likely_agri_counts.csv：县×年农业焚烧计数
- hlj_county_trend_summary.csv：县级 MK/Sen/Pettitt 摘要
- hlj_province_series.csv / .png：省级年度序列与图
- hlj_sen_slope_map.png：县域 Sen 斜率地图
- hlj_change_year_map.png：县域变点年份地图（无显著时为提示底图）
- top10_decreasing_counties.csv / top10_increasing_counties.csv：按 Sen 斜率排序的 Top 榜
- hlj_sen_slope_hist.png：县域 Sen 斜率分布直方图

备注：如需我将“季节化窗口趋势 + 预白化MK + bootstrap CI”落地为代码单元并生成补充图表，请告诉我优先级（省级或县级），我会直接新增并导出至 results/task3。

In [ ]:
# 任务3增强：Sen 斜率置信区间（支持90%与95%）+ 显著性地图 + 省级最佳切点
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
from itertools import combinations
from matplotlib import rcParams as _rc

# 设置中文字体
_rc['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'Microsoft YaHei', 'Noto Sans CJK SC'] + _rc.get('font.sans-serif', [])

# 开关：是否计算 Theil–Sen 置信区间与是否绘制显著性地图
COMPUTE_SEN_CI = False   # 用户反馈90%仍全灰 -> 默认不再计算CI（可改为True开启）
SHOW_CI_SIGNIFICANCE_MAP = False  # 默认不绘制显著性地图（可改为True开启）

ROOT = Path.cwd()
if ROOT.name.lower() == 'script': 
    ROOT = ROOT.parent
TASK3 = ROOT / 'results' / 'task3'
TASK3.mkdir(parents=True, exist_ok=True)

# 读取数据
cnt_full_path = TASK3 / 'hlj_county_year_likely_agri_counts.csv'
if not cnt_full_path.exists():
    raise FileNotFoundError(f"找不到文件: {cnt_full_path}")
cnt_full = pd.read_csv(cnt_full_path)

trend_path = TASK3 / 'hlj_county_trend_summary.csv'
if not trend_path.exists():
    raise FileNotFoundError(f"找不到文件: {trend_path}")
trend = pd.read_csv(trend_path)

# 1) Theil–Sen 斜率的经验置信区间
if COMPUTE_SEN_CI:
    CI_LEVELS = [0.90, 0.95]
    rows = []
    for code, g in cnt_full.groupby('county_code'):
        g = g.sort_values('year')
        y = g['year'].values
        v = g['count'].astype(float).values
        slopes = []
        for (i, j) in combinations(range(len(y)), 2):
            if y[j] == y[i]:
                continue
            slopes.append((v[j] - v[i]) / (y[j] - y[i]))
        rec = {'county_code': code}
        if len(slopes) == 0:
            rec.update({'sen_slope_med': 0.0})
            for lvl in CI_LEVELS:
                rec.update({f'sen_slope_lo_{int(lvl*100)}': 0.0, f'sen_slope_hi_{int(lvl*100)}': 0.0})
        else:
            slopes = np.array(slopes, dtype=float)
            rec['sen_slope_med'] = float(np.median(slopes))
            for lvl in CI_LEVELS:
                alpha = 1.0 - lvl
                lo = float(np.percentile(slopes, (alpha/2)*100.0))
                hi = float(np.percentile(slopes, (1.0 - alpha/2)*100.0))
                rec.update({f'sen_slope_lo_{int(lvl*100)}': lo, f'sen_slope_hi_{int(lvl*100)}': hi})
        rows.append(rec)
    sen_ci = pd.DataFrame(rows)
    enh = trend.merge(sen_ci, on='county_code', how='left')

    def _sig_by_cols(row, lo_col, hi_col):
        hi = row.get(hi_col, np.nan)
        lo = row.get(lo_col, np.nan)
        if pd.notna(hi) and hi < 0:
            return 'significant_decrease'
        if pd.notna(lo) and lo > 0:
            return 'significant_increase'
        return 'not_significant'

    enh['sen_significance_95'] = enh.apply(lambda r: _sig_by_cols(r, 'sen_slope_lo_95', 'sen_slope_hi_95'), axis=1)
    enh['sen_significance_90'] = enh.apply(lambda r: _sig_by_cols(r, 'sen_slope_lo_90', 'sen_slope_hi_90'), axis=1)
    # 兼容旧列名
    if 'sen_slope_lo_95' in enh.columns:
        enh['sen_slope_lo'] = enh['sen_slope_lo_95']
        enh['sen_slope_hi'] = enh['sen_slope_hi_95']
    enh['sen_significance'] = enh.get('sen_significance_90', 'not_significant')
else:
    # 不计算CI：直接把 trend 作为增强版导出
    enh = trend.copy()

enh_path = TASK3 / 'hlj_county_trend_summary_enhanced.csv'
enh.to_csv(enh_path, index=False, encoding='utf-8-sig')
print('保存:', enh_path)

# 2) 显著性地图
if SHOW_CI_SIGNIFICANCE_MAP and COMPUTE_SEN_CI:
    shp_dir = ROOT / 'raw_data' / 'chn_county'
    if not shp_dir.exists():
        raise FileNotFoundError(f"县界目录不存在: {shp_dir}")
    
    shp_files = list(shp_dir.glob('*.shp'))
    if not shp_files:
        raise FileNotFoundError(f"在 {shp_dir} 中未找到 .shp 文件")
    
    shp = shp_files[0]
    counties = gpd.read_file(shp)
    prov_col = '省级' if '省级' in counties.columns else ('NAME_1' if 'NAME_1' in counties.columns else None)
    if prov_col is None:
        possible_cols = ['province', 'PROVINCE', 'Province', 'name_1', 'NAME_1']
        for col in possible_cols:
            if col in counties.columns:
                prov_col = col
                break
        if prov_col is None:
            raise ValueError("无法找到省级列，请检查县界文件的列名")
        
    def _is_hlj(x):
        s = str(x)
        return ('黑龙江' in s) or ('Heilongjiang' in s) or ('Hēi Lóng Jiāng' in s)

    hlj_counties = counties[counties[prov_col].apply(_is_hlj)].copy()
    key = '县级码' if '县级码' in hlj_counties.columns else ('code' if 'code' in hlj_counties.columns else None)
    if key is None:
        possible_keys = ['COUNTY_CODE', 'adcode', 'ADCODE', 'County_Code', 'county_code', 'code']
        for col in possible_keys:
            if col in hlj_counties.columns:
                key = col
                break
        if key is None:
            raise ValueError("无法找到县级编码列，请检查县界文件的列名")

    gmap = hlj_counties.merge(enh, left_on=key, right_on='county_code', how='left')
    palette = {'significant_decrease': '#c44e52', 'significant_increase': '#4c72b0', 'not_significant': '#dddddd'}

    def _plot_sig_map(sig_col: str, title: str, out_name: str):
        fig, ax = plt.subplots(1, 1, figsize=(8, 6))
        for lab in ['not_significant', 'significant_decrease', 'significant_increase']:
            sub = gmap[gmap[sig_col] == lab]
            if len(sub) > 0:
                sub.plot(ax=ax, color=palette[lab], edgecolor='white', linewidth=0.1, label=lab.replace('_', ' '))
        ax.set_title(title)
        ax.legend(frameon=False, loc='lower left')
        png = TASK3 / out_name
        plt.savefig(png, dpi=180, bbox_inches='tight')
        plt.show()
        plt.close()
        print('保存:', png)

    _plot_sig_map('sen_significance_90', '黑龙江 县域 Theil–Sen 斜率90%置信区间显著性', 'hlj_sen_significance_map_ci90.png')
    _plot_sig_map('sen_significance_95', '黑龙江 县域 Theil–Sen 斜率95%置信区间显著性', 'hlj_sen_significance_map_ci95.png')
else:
    print('显著性地图已关闭（SHOW_CI_SIGNIFICANCE_MAP=False 或未计算CI）。')

# 3) 省级最佳切点（两段均值 SSE 最小）
prov = cnt_full.groupby('year')['count'].sum().reset_index().sort_values('year')
if len(prov) == 0:
    raise RuntimeError('省级年度序列数据为空，无法计算最佳切点。')

# 数据清洗和类型转换
prov['year'] = pd.to_numeric(prov['year'], errors='coerce')
prov['count'] = pd.to_numeric(prov['count'], errors='coerce')

# 移除空值
prov = prov.dropna(subset=['year', 'count'])

if len(prov) < 3:
    print('年度数据点不足，无法计算最佳切点。(至少需要3个年份)')
    best = {'year': None, 'sse': np.inf, 'pre_mean': np.nan, 'post_mean': np.nan, 'delta': np.nan}
else:
    years = prov['year'].values
    vals = prov['count'].values
    
    # 确保数据是数值类型
    years = years.astype(float)
    vals = vals.astype(float)
    
    best = {'year': None, 'sse': np.inf, 'pre_mean': np.nan, 'post_mean': np.nan, 'delta': np.nan}

    for k in range(1, len(years)-1):
        pre = vals[:k]
        post = vals[k:]
        
        # 确保数据是 numpy array 并且没有空值
        pre = np.array(pre, dtype=float)
        post = np.array(post, dtype=float)
        
        if len(pre) > 0 and len(post) > 0 and not np.any(np.isnan(pre)) and not np.any(np.isnan(post)):
            pre_mean = np.mean(pre)
            post_mean = np.mean(post)
            sse = np.sum((pre - pre_mean)**2) + np.sum((post - post_mean)**2)
            
            if sse < best['sse']:
                best.update({
                    'year': int(years[k]),
                    'sse': float(sse),
                    'pre_mean': float(pre_mean),
                    'post_mean': float(post_mean),
                    'delta': float(post_mean - pre_mean)
                })

best_df = pd.DataFrame([best])
best_path = TASK3 / 'hlj_province_best_split.csv'
best_df.to_csv(best_path, index=False, encoding='utf-8-sig')
print('保存:', best_path)

# 绘图
fig, ax = plt.subplots(1, 1, figsize=(7, 4))

if len(prov) > 0:
    # 确保数据是数值类型
    plot_years = prov['year'].values.astype(float)
    plot_vals = prov['count'].values.astype(float)
    
    ax.plot(plot_years, plot_vals, marker='o')
    
    if best['year'] is not None:
        ax.axvline(best['year'], color='red', ls='--', label=f"Best split {best['year']}")
        
    ax.set_title('黑龙江 省级年度序列与最佳切点（两段均值）')
    ax.set_xlabel('Year')
    ax.set_ylabel('Count')
    ax.legend(loc='upper left')
else:
    ax.text(0.5, 0.5, 'No data available', ha='center', va='center', transform=ax.transAxes)
    print('省级年度序列数据为空，无法绘制图表。')

png = TASK3 / 'hlj_province_series_best_split.png'
plt.savefig(png, dpi=180, bbox_inches='tight')
plt.show()
plt.close()
print('保存:', png)